In [ ]:
from google.colab import files
uploaded = files.upload()

Saving merged_energy_weather_data.csv to merged_energy_weather_data.csv


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
import pandas as pd

# Load and rename columns
df = pd.read_csv("merged_energy_weather_data.csv", parse_dates=["Date"])
df.columns = [
    "date", "price", "temp", "precip", "wind", "humidity",
    "cloud", "radiation", "week_day", "month", "day_month"
]
print(df.shape)
df.head()

df = df.sort_values("date").reset_index(drop=True)
cloud_missing = df['cloud'].isna()
df.loc[cloud_missing, 'cloud'] = (
    df['cloud'].shift(1) + df['cloud'].shift(-1)
) / 2
print("Remaining NaNs:\n", df.isna().sum())

(3865, 11)
Remaining NaNs:
 date         0
price        0
temp         0
precip       0
wind         0
humidity     0
cloud        0
radiation    0
week_day     0
month        0
day_month    0
dtype: int64


In [ ]:
df = df.drop(columns=["day_month"])
for lag in range(1, 8):
    df[f"lag_{lag}"] = df["price"].shift(lag)
df.head(10)

,date,price,temp,precip,wind,humidity,cloud,radiation,week_day,month,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7
0,2015-01-01,14.706087,6.440000,3.060714,8.392733,93.472826,95.130383,83.391304,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-02,7.075217,6.490322,5.457143,12.129323,84.318486,70.348317,456.913043,4,1,14.706087,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-03,15.683333,4.429734,4.339286,8.461127,83.168743,35.944177,503.043478,5,1,7.075217,14.706087,NaN,NaN,NaN,NaN,NaN
3,2015-01-04,23.827727,3.844092,1.325000,6.188518,79.221513,30.214925,705.652174,6,1,15.683333,7.075217,14.706087,NaN,NaN,NaN,NaN
4,2015-01-05,38.073750,4.910970,0.978571,3.414181,92.058873,77.380103,239.478261,0,1,23.827727,15.683333,7.075217,14.706087,NaN,NaN,NaN
5,2015-01-06,29.161739,4.021349,0.810714,5.655600,92.616659,85.929154,110.086957,1,1,38.073750,23.827727,15.683333,7.075217,14.706087,NaN,NaN
6,2015-01-07,29.011667,4.691019,1.192857,5.781492,91.003623,54.440213,622.695652,2,1,29.161739,38.073750,23.827727,15.683333,7.075217,14.706087,NaN
7,2015-01-08,24.343478,6.013413,3.214286,7.359274,92.091256,71.431795,288.086957,3,1,29.011667,29.161739,38.073750,23.827727,15.683333,7.075217,14.706087
8,2015-01-09,23.687917,5.783474,9.492857,10.275274,81.372204,72.396564,270.913043,4,1,24.343478,29.011667,29.161739,38.073750,23.827727,15.683333,7.075217
9,2015-01-10,14.021250,6.319171,7.303571,11.792501,83.443880,73.162782,236.913043,5,1,23.687917,24.343478,29.011667,29.161739,38.073750,23.827727,15.683333


In [ ]:
df = df.dropna().reset_index(drop=True)

print(df.shape)
df.head()

(3858, 17)


,date,price,temp,precip,wind,humidity,cloud,radiation,week_day,month,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7
0,2015-01-08,24.343478,6.013413,3.214286,7.359274,92.091256,71.431795,288.086957,3,1,29.011667,29.161739,38.073750,23.827727,15.683333,7.075217,14.706087
1,2015-01-09,23.687917,5.783474,9.492857,10.275274,81.372204,72.396564,270.913043,4,1,24.343478,29.011667,29.161739,38.073750,23.827727,15.683333,7.075217
2,2015-01-10,14.021250,6.319171,7.303571,11.792501,83.443880,73.162782,236.913043,5,1,23.687917,24.343478,29.011667,29.161739,38.073750,23.827727,15.683333
3,2015-01-11,4.296087,3.989937,2.457143,10.834715,79.724570,42.435521,406.434783,6,1,14.021250,23.687917,24.343478,29.011667,29.161739,38.073750,23.827727
4,2015-01-12,10.938696,6.529484,2.867857,9.851411,87.450542,69.666470,123.043478,0,1,4.296087,14.021250,23.687917,24.343478,29.011667,29.161739,38.073750


**Feedforward NN**

In [ ]:
import random
import numpy as np
import torch

def set_seed(seed=123):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(123)

In [ ]:
import numpy as np

forecast_horizon = 14

# Feature matrix: all columns except target and date
feature_cols = df.columns.difference(['date', 'price']).tolist()
feature_cols.sort()

X = df[feature_cols].values

# Create rolling 14-day future targets
y = []
for i in range(len(df) - forecast_horizon):
    y.append(df['price'].iloc[i+1 : i+1+forecast_horizon].values)

X = X[:len(y)]
y = np.array(y)

In [ ]:
def expanding_window_cv(X, y, initial_train_size=1095, horizon=14, step=14):
    """
    Generator yielding expanding window splits with one 14-day forecast per fold.
    """
    n_samples = len(X)
    start = initial_train_size

    while start + 1 <= n_samples:
        X_train = X[:start]
        y_train = y[:start]
        X_val = X[start:start + 1]  # Single sample (14-day forecast)
        y_val = y[start:start + 1]
        yield X_train, y_train, X_val, y_val
        start += step

In [ ]:
for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
    expanding_window_cv(X, y, initial_train_size=1095, horizon=14, step=14)
):
    print(f"Fold {fold+1}")
    print(f"  Train shape: {X_tr.shape}, {y_tr.shape}")
    print(f"  Val shape:   {X_val.shape}, {y_val.shape}")

Fold 1
  Train shape: (1095, 15), (1095, 14)
  Val shape:   (1, 15), (1, 14)
Fold 2
  Train shape: (1109, 15), (1109, 14)
  Val shape:   (1, 15), (1, 14)
Fold 3
  Train shape: (1123, 15), (1123, 14)
  Val shape:   (1, 15), (1, 14)
Fold 4
  Train shape: (1137, 15), (1137, 14)
  Val shape:   (1, 15), (1, 14)
Fold 5
  Train shape: (1151, 15), (1151, 14)
  Val shape:   (1, 15), (1, 14)
Fold 6
  Train shape: (1165, 15), (1165, 14)
  Val shape:   (1, 15), (1, 14)
Fold 7
  Train shape: (1179, 15), (1179, 14)
  Val shape:   (1, 15), (1, 14)
Fold 8
  Train shape: (1193, 15), (1193, 14)
  Val shape:   (1, 15), (1, 14)
Fold 9
  Train shape: (1207, 15), (1207, 14)
  Val shape:   (1, 15), (1, 14)
Fold 10
  Train shape: (1221, 15), (1221, 14)
  Val shape:   (1, 15), (1, 14)
Fold 11
  Train shape: (1235, 15), (1235, 14)
  Val shape:   (1, 15), (1, 14)
Fold 12
  Train shape: (1249, 15), (1249, 14)
  Val shape:   (1, 15), (1, 14)
Fold 13
  Train shape: (1263, 15), (1263, 14)
  Val shape:   (1, 15), (1,

In [ ]:
from sklearn.preprocessing import StandardScaler

# Use first fold
X_train, y_train, X_val, y_val = next(expanding_window_cv(X, y))

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled = scaler_X.transform(X_val)

y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled = scaler_y.transform(y_val)

print("Scaled shapes:")
print("  X_train_scaled:", X_train_scaled.shape)
print("  X_val_scaled:  ", X_val_scaled.shape)
print("  y_train_scaled:", y_train_scaled.shape)
print("  y_val_scaled:  ", y_val_scaled.shape)

Scaled shapes:
  X_train_scaled: (1095, 15)
  X_val_scaled:   (1, 15)
  y_train_scaled: (1095, 14)
  y_val_scaled:   (1, 14)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class PriceForecastDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
import torch.nn as nn

class FeedforwardNN(nn.Module):
    def __init__(self, input_dim, hidden_dims=[64, 32], dropout=0.2):
        super().__init__()
        layers = []
        dims = [input_dim] + hidden_dims
        for i in range(len(dims) - 1):
            layers.append(nn.Linear(dims[i], dims[i + 1]))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
        layers.append(nn.Linear(dims[-1], 14))  # 14-day forecast
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

def train_one_fold(X_train, y_train, X_val, y_val, epochs=10, lr=0.001, batch_size=32):
    # Scale input/output
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train_scaled = scaler_X.fit_transform(X_train)
    X_val_scaled = scaler_X.transform(X_val)

    y_train_scaled = scaler_y.fit_transform(y_train)
    y_val_scaled = scaler_y.transform(y_val)

    # Datasets and DataLoaders
    train_dataset = PriceForecastDataset(X_train_scaled, y_train_scaled)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    # Model setup
    model = FeedforwardNN(input_dim=X_train.shape[1]).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    # Training
    model.train()
    for epoch in range(epochs):
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            preds = model(X_batch)
            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)
        preds_scaled = model(X_val_tensor).cpu().numpy()
        preds = scaler_y.inverse_transform(preds_scaled)
        actuals = scaler_y.inverse_transform(y_val_scaled)

    # Metrics
    mae = mean_absolute_error(actuals.flatten(), preds.flatten())
    mse = mean_squared_error(actuals.flatten(), preds.flatten())
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((actuals - preds) / np.abs(actuals))) * 100

    return mae, rmse, mape

In [ ]:
mae_scores = []
rmse_scores = []
mape_scores = []

for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
    expanding_window_cv(X, y, initial_train_size=1095, horizon=14, step=14)
):
    print(f"Fold {fold+1}")
    mae, rmse, mape = train_one_fold(X_tr, y_tr, X_val, y_val)
    print(f"  MAE: {mae:.4f}, RMSE: {rmse:.4f}, MAPE: {mape:.4f}")
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    mape_scores.append(mape)

print("\n Final CV Results:")
print(f"Average MAE:  {np.mean(mae_scores):.4f}")
print(f"Average RMSE: {np.mean(rmse_scores):.4f}")
print(f"Average MAPE: {np.mean(mape_scores):.4f}")

Fold 1
  MAE: 6.2473, RMSE: 7.1531, MAPE: 17.5927
Fold 2
  MAE: 5.3045, RMSE: 6.6728, MAPE: 23.4663
Fold 3
  MAE: 8.5751, RMSE: 10.5162, MAPE: 27.8835
Fold 4
  MAE: 11.5512, RMSE: 12.7947, MAPE: 26.8518
Fold 5
  MAE: 10.6407, RMSE: 12.8441, MAPE: 110.9050
Fold 6
  MAE: 10.5127, RMSE: 11.1842, MAPE: 24.9258
Fold 7
  MAE: 4.7153, RMSE: 5.7063, MAPE: 14.6408
Fold 8
  MAE: 5.2797, RMSE: 6.4927, MAPE: 13.4753
Fold 9
  MAE: 7.1043, RMSE: 8.4423, MAPE: 31.3333
Fold 10
  MAE: 8.0454, RMSE: 8.9329, MAPE: 20.8324
Fold 11
  MAE: 12.4983, RMSE: 12.8175, MAPE: 26.7261
Fold 12
  MAE: 7.6326, RMSE: 8.0471, MAPE: 19.5679
Fold 13
  MAE: 16.2515, RMSE: 17.1699, MAPE: 32.0438
Fold 14
  MAE: 5.9484, RMSE: 6.3339, MAPE: 11.3156
Fold 15
  MAE: 4.9652, RMSE: 5.9237, MAPE: 8.7144
Fold 16
  MAE: 5.1129, RMSE: 6.4292, MAPE: 9.3011
Fold 17
  MAE: 7.9587, RMSE: 9.4586, MAPE: 12.9831
Fold 18
  MAE: 8.0411, RMSE: 9.2881, MAPE: 13.2988
Fold 19
  MAE: 10.0517, RMSE: 12.6109, MAPE: 33.5137
Fold 20
  MAE: 14.5324, RMSE

In [ ]:
# with early stopping
def train_one_fold(X_train, y_train, X_val, y_val,
                   epochs=50, lr=0.001, batch_size=32, patience=5):
    torch.manual_seed(123)

    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    import numpy as np

    # Scale inputs
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train_scaled = scaler_X.fit_transform(X_train)
    X_val_scaled = scaler_X.transform(X_val)

    y_train_scaled = scaler_y.fit_transform(y_train)
    y_val_scaled = scaler_y.transform(y_val)

    # Dataset & DataLoader
    train_dataset = PriceForecastDataset(X_train_scaled, y_train_scaled)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    # Model setup
    model = FeedforwardNN(input_dim=X_train.shape[1]).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    criterion = nn.MSELoss()

    # Early stopping setup
    best_val_loss = float('inf')
    best_model_state = None
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            preds = model(X_batch)
            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Validation loss (on 1 val sample)
        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)
            y_val_tensor = torch.tensor(y_val_scaled, dtype=torch.float32).to(device)
            val_preds = model(X_val_tensor)
            val_loss = criterion(val_preds, y_val_tensor).item()

        # Early stopping check
        if val_loss < best_val_loss - 1e-4:
            best_val_loss = val_loss
            best_model_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # Restore best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    # Final prediction
    model.eval()
    with torch.no_grad():
        X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)
        preds_scaled = model(X_val_tensor).cpu().numpy()
        preds = scaler_y.inverse_transform(preds_scaled)
        actuals = scaler_y.inverse_transform(y_val_scaled)

    mae = mean_absolute_error(actuals.flatten(), preds.flatten())
    rmse = np.sqrt(mean_squared_error(actuals.flatten(), preds.flatten()))
    mape = np.mean(np.abs((actuals - preds) / (actuals + 1e-8))) * 100

    return mae, rmse, mape

In [ ]:
mae_scores = []
rmse_scores = []
mape_scores = []

for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
    expanding_window_cv(X, y, initial_train_size=1095, horizon=14, step=14)
):
    print(f"Fold {fold+1}")
    mae, rmse, mape = train_one_fold(X_tr, y_tr, X_val, y_val)
    print(f"  MAE: {mae:.4f}, RMSE: {rmse:.4f}, MAPE: {mape:.4f}")
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    mape_scores.append(mape)

print("\n Final CV Results:")
print(f"Average MAE:  {np.mean(mae_scores):.4f}")
print(f"Average RMSE: {np.mean(rmse_scores):.4f}")
print(f"Average MAPE: {np.mean(mape_scores):.4f}")

Fold 1
⏹️ Early stopping at epoch 33
  MAE: 5.4730, RMSE: 6.3111, MAPE: 15.4720
Fold 2
⏹️ Early stopping at epoch 27
  MAE: 5.2818, RMSE: 6.5054, MAPE: 23.4355
Fold 3
⏹️ Early stopping at epoch 46
  MAE: 7.3530, RMSE: 8.9523, MAPE: 25.3389
Fold 4
⏹️ Early stopping at epoch 14
  MAE: 11.2930, RMSE: 12.4842, MAPE: 26.2442
Fold 5
⏹️ Early stopping at epoch 14
  MAE: 10.8048, RMSE: 13.2135, MAPE: 115.7303
Fold 6
⏹️ Early stopping at epoch 23
  MAE: 9.8491, RMSE: 10.6902, MAPE: 23.3322
Fold 7
⏹️ Early stopping at epoch 12
  MAE: 5.2988, RMSE: 6.1739, MAPE: 15.9145
Fold 8
⏹️ Early stopping at epoch 11
  MAE: 5.9077, RMSE: 6.8902, MAPE: 15.2768
Fold 9
⏹️ Early stopping at epoch 8
  MAE: 6.9886, RMSE: 8.2625, MAPE: 30.5973
Fold 10
⏹️ Early stopping at epoch 39
  MAE: 7.7743, RMSE: 8.6949, MAPE: 19.9236
Fold 11
⏹️ Early stopping at epoch 23
  MAE: 12.6153, RMSE: 12.8803, MAPE: 27.0893
Fold 12
⏹️ Early stopping at epoch 11
  MAE: 7.5123, RMSE: 8.0771, MAPE: 19.9018
Fold 13
⏹️ Early stopping at e



Hyperparameter tunning

In [ ]:
def train_one_fold(X_train, y_train, X_val, y_val,
                   hidden_dims=[64, 32], dropout=0.2,
                   epochs=50, lr=0.001, batch_size=32, patience=5):

    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import mean_absolute_error, mean_squared_error

    # Scale input/output
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train_scaled = scaler_X.fit_transform(X_train)
    X_val_scaled = scaler_X.transform(X_val)

    y_train_scaled = scaler_y.fit_transform(y_train)
    y_val_scaled = scaler_y.transform(y_val)

    # PyTorch dataset and DataLoader
    train_dataset = PriceForecastDataset(X_train_scaled, y_train_scaled)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    # Model
    model = FeedforwardNN(input_dim=X_train.shape[1], hidden_dims=hidden_dims, dropout=dropout).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    criterion = nn.MSELoss()

    # Early stopping
    best_val_loss = float('inf')
    best_model_state = None
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            preds = model(X_batch)
            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)
            y_val_tensor = torch.tensor(y_val_scaled, dtype=torch.float32).to(device)
            val_preds = model(X_val_tensor)
            val_loss = criterion(val_preds, y_val_tensor).item()

        if val_loss < best_val_loss - 1e-4:
            best_val_loss = val_loss
            best_model_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
         #       print(f"Early stopping at epoch {epoch+1}")
                break

    # Load best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    # Final prediction
    model.eval()
    with torch.no_grad():
        X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)
        preds_scaled = model(X_val_tensor).cpu().numpy()
        preds = scaler_y.inverse_transform(preds_scaled)
        actuals = scaler_y.inverse_transform(y_val_scaled)

    # Metrics
    mae = mean_absolute_error(actuals.flatten(), preds.flatten())
    rmse = np.sqrt(mean_squared_error(actuals.flatten(), preds.flatten()))
    mape = np.mean(np.abs((actuals - preds) / (actuals + 1e-8))) * 100

    return mae, rmse, mape

In [ ]:
import itertools

# Define hyperparameter options
hidden_layer_options = [[64, 32], [32, 16], [128, 64, 32], [64]]
dropout_options = [0.0, 0.2, 0.3]
lr_options = [0.001, 0.0005]

# All combinations
param_grid = list(itertools.product(hidden_layer_options, dropout_options, lr_options))

In [ ]:
results = []

for i, (hidden_dims, dropout, lr) in enumerate(param_grid):
    print(f"\n Config {i+1}/{len(param_grid)}: hidden={hidden_dims}, dropout={dropout}, lr={lr}")

    mae_scores = []
    rmse_scores = []
    mape_scores = []

    for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
        expanding_window_cv(X, y, initial_train_size=1095, horizon=14, step=14)
    ):
        mae, rmse, mape = train_one_fold(
            X_tr, y_tr, X_val, y_val,
            hidden_dims=hidden_dims,
            dropout=dropout,
            lr=lr,
            epochs=50,
            patience=5,
            batch_size=32
        )
        mae_scores.append(mae)
        rmse_scores.append(rmse)
        mape_scores.append(mape)

    avg_mae = np.mean(mae_scores)
    avg_rmse = np.mean(rmse_scores)
    avg_mape = np.mean(mape_scores)

    results.append({
        "hidden_dims": hidden_dims,
        "dropout": dropout,
        "lr": lr,
        "MAE": avg_mae,
        "RMSE": avg_rmse,
        "MAPE": avg_mape
    })

    print(f"Avg MAE: {avg_mae:.4f}, RMSE: {avg_rmse:.4f}, MAPE: {avg_mape:.4f}")


 Config 1/24: hidden=[64, 32], dropout=0.0, lr=0.001
Avg MAE: 27.3507, RMSE: 32.9551, MAPE: 80.8614

 Config 2/24: hidden=[64, 32], dropout=0.0, lr=0.0005
Avg MAE: 26.9973, RMSE: 32.5369, MAPE: 81.0100

 Config 3/24: hidden=[64, 32], dropout=0.2, lr=0.001
Avg MAE: 27.6666, RMSE: 33.2643, MAPE: 81.7499

 Config 4/24: hidden=[64, 32], dropout=0.2, lr=0.0005
Avg MAE: 27.6302, RMSE: 33.1489, MAPE: 79.0938

 Config 5/24: hidden=[64, 32], dropout=0.3, lr=0.001
Avg MAE: 27.7590, RMSE: 33.1863, MAPE: 80.5522

 Config 6/24: hidden=[64, 32], dropout=0.3, lr=0.0005
Avg MAE: 27.8262, RMSE: 33.4035, MAPE: 84.2523

 Config 7/24: hidden=[32, 16], dropout=0.0, lr=0.001
Avg MAE: 27.1601, RMSE: 32.7037, MAPE: 82.5618

 Config 8/24: hidden=[32, 16], dropout=0.0, lr=0.0005
Avg MAE: 27.3554, RMSE: 33.0033, MAPE: 79.9574

 Config 9/24: hidden=[32, 16], dropout=0.2, lr=0.001
Avg MAE: 28.2018, RMSE: 33.7014, MAPE: 80.8394

 Config 10/24: hidden=[32, 16], dropout=0.2, lr=0.0005
Avg MAE: 27.9062, RMSE: 33.5329

In [ ]:
import pandas as pd

df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by="RMSE").reset_index(drop=True)
df_results

,hidden_dims,dropout,lr,MAE,RMSE,MAPE
0,[64],0.0,0.0010,26.682077,32.237810,80.940988
1,"[64, 32]",0.0,0.0005,26.997264,32.536932,81.009969
2,[64],0.0,0.0005,26.963289,32.597032,82.960948
3,"[32, 16]",0.0,0.0010,27.160058,32.703665,82.561809
4,[64],0.3,0.0010,27.413612,32.844114,80.201353
5,[64],0.2,0.0010,27.325231,32.864061,80.702280
6,[64],0.2,0.0005,27.460403,32.924670,80.537835
7,"[64, 32]",0.0,0.0010,27.350739,32.955068,80.861363
8,"[32, 16]",0.0,0.0005,27.355444,33.003290,79.957396
9,"[128, 64, 32]",0.0,0.0005,27.445917,33.005884,81.033842


**LSTM**

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import trange

# --- 1. LSTM Model ---
class LSTMForecastNet(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=1, dropout=0.2, output_size=14):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                            batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # [batch, seq, hidden]
        out = self.fc(lstm_out[:, -1, :])  # [batch, output_size]
        return out

In [ ]:
# --- 2. Dataset Class  ---
class LSTMForecastDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)  # [samples, seq_len, features]
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
# --- 3. Sliding Window Input Creation ---
def create_lstm_input(X_scaled, y_scaled, sequence_len=30):
    X_lstm = []
    y_lstm = []
    for i in range(sequence_len, len(X_scaled)):
        X_window = X_scaled[i-sequence_len:i]
        y_target = y_scaled[i]  # already a 14-element array
        X_lstm.append(X_window)
        y_lstm.append(y_target)
    return np.array(X_lstm), np.array(y_lstm)

In [ ]:
# --- 4. Training One Fold ---
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import numpy as np
import torch
from torch.utils.data import DataLoader

def train_one_fold(X_train, y_train, X_val, y_val,
                   sequence_len=30, num_epochs=50, patience=5,
                   batch_size=32, lr=0.001, hidden_size=64, num_layers=1, dropout=0.2):

    # --- 1. Scaling ---
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train_scaled = scaler_X.fit_transform(X_train)
    X_val_scaled = scaler_X.transform(X_val)

    y_train_scaled = scaler_y.fit_transform(y_train)  # shape: (n_samples, 14)
    y_val_scaled = scaler_y.transform(y_val)

    # --- 2. Create LSTM sequences ---
    X_train_seq, y_train_seq = create_lstm_input(X_train_scaled, y_train_scaled, sequence_len)
    X_val_seq, y_val_seq = create_lstm_input(X_val_scaled, y_val_scaled, sequence_len)

    if X_val_seq.shape[0] == 0:
        print("Skipping fold: Not enough validation sequences.")
        return None, None, None

    # --- 3. Datasets and Loaders ---
    train_loader = DataLoader(
        LSTMForecastDataset(X_train_seq, y_train_seq), batch_size=batch_size, shuffle=False
    )
    val_loader = DataLoader(
        LSTMForecastDataset(X_val_seq, y_val_seq), batch_size=batch_size, shuffle=False
    )

    # --- 4. Model setup ---
    model = LSTMForecastNet(
        input_size=X_train_seq.shape[2],
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout=dropout
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    # --- 5. Training Loop with Early Stopping ---
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None

    for epoch in range(num_epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)

            optimizer.zero_grad()
            preds = model(xb)  # shape: (batch_size, 14)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            val_preds = torch.cat([model(xb.to(device)) for xb, _ in val_loader])
            val_targets = torch.cat([yb.to(device) for _, yb in val_loader])
            val_loss = criterion(val_preds, val_targets)

        # Early stopping
        if val_loss.item() < best_val_loss - 1e-4:
            best_val_loss = val_loss.item()
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break # Early stopping is triggered here

    # --- 6. Load best model ---
    if best_model_state:
        model.load_state_dict(best_model_state)

    # --- 7. Final Evaluation ---
    model.eval()
    with torch.no_grad():
        X_val_tensor = torch.tensor(X_val_seq, dtype=torch.float32).to(device)
        preds_scaled = model(X_val_tensor).cpu().numpy()
        targets_scaled = y_val_seq  # already NumPy array

    # Inverse scaling
    preds = scaler_y.inverse_transform(preds_scaled)
    targets = scaler_y.inverse_transform(targets_scaled)

    # --- 8. Metrics ---
    mae = mean_absolute_error(targets.flatten(), preds.flatten())
    rmse = np.sqrt(mean_squared_error(targets.flatten(), preds.flatten()))
    mape = mean_absolute_percentage_error(targets.flatten(), preds.flatten()) * 100

    return mae, rmse, mape

In [ ]:
# --- 5. Expanding Window Cross Validation ---
def expanding_window_cv(X, y, initial_train_size, horizon, step, sequence_len):
    # Ensure we have enough data for multiple sequences in validation
    min_val_size = 60  # Should be > sequence_len + horizon to allow multiple samples
    max_idx = len(X)

    for start in range(initial_train_size, max_idx - min_val_size + 1, step):
        end_val = start + min_val_size
        if end_val > max_idx:
            break
        X_tr, y_tr = X[:start], y[:start]
        X_val, y_val = X[start:end_val], y[start:end_val]
        yield X_tr, y_tr, X_val, y_val

In [ ]:
# --- 6. Main CV Execution ---
import warnings
warnings.filterwarnings("ignore", message="dropout option adds dropout after all but last recurrent layer")
warnings.filterwarnings("ignore", message="To copy construct from a tensor")

sequence_len = 30
horizon = 14
initial_train_size = 1095  # 3 years

mae_scores = []
rmse_scores = []
mape_scores = []

for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
    expanding_window_cv(X, y, initial_train_size=initial_train_size, horizon=horizon, step=horizon, sequence_len=sequence_len)
):
    print(f"\nFold {fold+1}")
    mae, rmse, mape = train_one_fold(
        X_tr, y_tr, X_val, y_val, sequence_len=sequence_len
    )
    if mae is None:
        continue  # Skip if fold was invalid
    print(f"  MAE: {mae:.4f}, RMSE: {rmse:.4f}, MAPE: {mape:.4f}")
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    mape_scores.append(mape)

# --- 7. Final Results ---
print("\n Final CV Results:")
print(f"Average MAE:  {np.mean(mae_scores):.4f}")
print(f"Average RMSE: {np.mean(rmse_scores):.4f}")
print(f"Average MAPE: {np.mean(mape_scores):.4f}")


Fold 1
  MAE: 10.5256, RMSE: 12.0631, MAPE: 39.04

Fold 2
  MAE: 10.1975, RMSE: 11.6274, MAPE: 56.95

Fold 3
  MAE: 7.9343, RMSE: 9.4685, MAPE: 48.71

Fold 4
  MAE: 5.5035, RMSE: 6.4717, MAPE: 15.51

Fold 5
  MAE: 5.3425, RMSE: 6.9173, MAPE: 18.99

Fold 6
  MAE: 6.4140, RMSE: 8.1413, MAPE: 24.17

Fold 7
  MAE: 9.6063, RMSE: 10.7867, MAPE: 25.57

Fold 8
  MAE: 10.9263, RMSE: 11.6769, MAPE: 24.53

Fold 9
  MAE: 11.5167, RMSE: 12.4942, MAPE: 24.96

Fold 10
  MAE: 12.1055, RMSE: 12.9934, MAPE: 24.87

Fold 11
  MAE: 15.8082, RMSE: 16.3377, MAPE: 29.60

Fold 12
  MAE: 13.2002, RMSE: 14.4323, MAPE: 23.76

Fold 13
  MAE: 10.9116, RMSE: 12.2685, MAPE: 18.87

Fold 14
  MAE: 16.2999, RMSE: 17.7054, MAPE: 27.40

Fold 15
  MAE: 10.4617, RMSE: 12.7986, MAPE: 24.28

Fold 16
  MAE: 11.3920, RMSE: 14.1156, MAPE: 33.23

Fold 17
  MAE: 13.1145, RMSE: 16.0565, MAPE: 32.65

Fold 18
  MAE: 11.0997, RMSE: 14.1720, MAPE: 23.12

Fold 19
  MAE: 11.8058, RMSE: 14.6532, MAPE: 21.82

Fold 20
  MAE: 14.5731, RMSE:

Hyperparamter tunning 1

In [ ]:
import itertools
import pandas as pd

# Define hyperparameter grid
hidden_sizes = [32, 64, 128]
num_layers_list = [1, 2, 3]
dropouts = [0.0, 0.1, 0.2, 0.3]
learning_rates = [0.001] # subsistute with 0.0005
batch_sizes = [32] # substitute with 64
sequence_lens = [30]

# Create grid of all combinations
param_grid = list(itertools.product(
    hidden_sizes,
    num_layers_list,
    dropouts,
    learning_rates,
    batch_sizes,
    sequence_lens
))

print(f"Total combinations: {len(param_grid)}")

🔍 Total combinations: 36


In [ ]:
results = []

for i, (hidden_size, num_layers, dropout, lr, batch_size, sequence_len) in enumerate(param_grid):
    print(f"\n Config {i+1}/{len(param_grid)}: hidden={hidden_size}, layers={num_layers}, dropout={dropout}, "
          f"lr={lr}, batch_size={batch_size}, seq_len={sequence_len}")

    mae_scores = []
    rmse_scores = []
    mape_scores = []

    for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
        expanding_window_cv(X, y,
                            initial_train_size=1095,
                            horizon=14,
                            step=14,
                            sequence_len=sequence_len)
    ):
        mae, rmse, mape = train_one_fold(
            X_tr, y_tr, X_val, y_val,
            sequence_len=sequence_len,
            num_epochs=50,
            patience=5,
            batch_size=batch_size,
            lr=lr,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout
        )

        if mae is None:
            continue  # skip invalid fold

        mae_scores.append(mae)
        rmse_scores.append(rmse)
        mape_scores.append(mape)

    if mae_scores:  # only record results if at least one fold succeeded
        results.append({
            "hidden_size": hidden_size,
            "num_layers": num_layers,
            "dropout": dropout,
            "lr": lr,
            "batch_size": batch_size,
            "sequence_len": sequence_len,
            "MAE": np.mean(mae_scores),
            "RMSE": np.mean(rmse_scores),
            "MAPE": np.mean(mape_scores)
        })


🔧 Config 1/36: hidden=32, layers=1, dropout=0.0, lr=0.001, batch_size=32, seq_len=30

🔧 Config 2/36: hidden=32, layers=1, dropout=0.1, lr=0.001, batch_size=32, seq_len=30

🔧 Config 3/36: hidden=32, layers=1, dropout=0.2, lr=0.001, batch_size=32, seq_len=30

🔧 Config 4/36: hidden=32, layers=1, dropout=0.3, lr=0.001, batch_size=32, seq_len=30

🔧 Config 5/36: hidden=32, layers=2, dropout=0.0, lr=0.001, batch_size=32, seq_len=30

🔧 Config 6/36: hidden=32, layers=2, dropout=0.1, lr=0.001, batch_size=32, seq_len=30

🔧 Config 7/36: hidden=32, layers=2, dropout=0.2, lr=0.001, batch_size=32, seq_len=30

🔧 Config 8/36: hidden=32, layers=2, dropout=0.3, lr=0.001, batch_size=32, seq_len=30

🔧 Config 9/36: hidden=32, layers=3, dropout=0.0, lr=0.001, batch_size=32, seq_len=30

🔧 Config 10/36: hidden=32, layers=3, dropout=0.1, lr=0.001, batch_size=32, seq_len=30

🔧 Config 11/36: hidden=32, layers=3, dropout=0.2, lr=0.001, batch_size=32, seq_len=30

🔧 Config 12/36: hidden=32, layers=3, dropout=0.3, l

In [ ]:
df_results = pd.DataFrame(results)
df_results = df_results.sort_values("RMSE").reset_index(drop=True)
df_results

,hidden_size,num_layers,dropout,lr,batch_size,sequence_len,MAE,RMSE,MAPE
0,32,1,0.3,0.001,32,30,32.280940,39.277221,92.953494
1,64,1,0.3,0.001,32,30,32.253175,39.309602,94.525646
2,64,1,0.0,0.001,32,30,32.351470,39.368226,93.607509
3,32,1,0.2,0.001,32,30,32.369872,39.380065,93.521553
4,32,1,0.0,0.001,32,30,32.443834,39.389070,92.909008
5,64,1,0.2,0.001,32,30,32.419429,39.470682,93.462581
6,32,1,0.1,0.001,32,30,32.794088,39.782033,93.209023
7,128,1,0.0,0.001,32,30,32.874109,39.821238,93.028981
8,128,1,0.3,0.001,32,30,32.947169,39.893616,93.532364
9,128,1,0.1,0.001,32,30,32.908045,39.935081,96.058793


Hyperparameter tunning 2

In [ ]:
import itertools
import pandas as pd

# Define hyperparameter grid
hidden_sizes = [32, 64, 128]
num_layers_list = [1, 2, 3]
dropouts = [0.0, 0.1, 0.2, 0.3]
learning_rates = [0.0005]
batch_sizes = [32] # substitute with 64
sequence_lens = [30]

# Create grid of all combinations
param_grid = list(itertools.product(
    hidden_sizes,
    num_layers_list,
    dropouts,
    learning_rates,
    batch_sizes,
    sequence_lens
))

print(f"Total combinations: {len(param_grid)}")

🔍 Total combinations: 36


In [ ]:
results = []

for i, (hidden_size, num_layers, dropout, lr, batch_size, sequence_len) in enumerate(param_grid):
    print(f"\n Config {i+1}/{len(param_grid)}: hidden={hidden_size}, layers={num_layers}, dropout={dropout}, "
          f"lr={lr}, batch_size={batch_size}, seq_len={sequence_len}")

    mae_scores = []
    rmse_scores = []
    mape_scores = []

    for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
        expanding_window_cv(X, y,
                            initial_train_size=1095,
                            horizon=14,
                            step=14,
                            sequence_len=sequence_len)
    ):
        mae, rmse, mape = train_one_fold(
            X_tr, y_tr, X_val, y_val,
            sequence_len=sequence_len,
            num_epochs=50,
            patience=5,
            batch_size=batch_size,
            lr=lr,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout
        )

        if mae is None:
            continue  # skip invalid fold

        mae_scores.append(mae)
        rmse_scores.append(rmse)
        mape_scores.append(mape)

    if mae_scores:  # only record results if at least one fold succeeded
        results.append({
            "hidden_size": hidden_size,
            "num_layers": num_layers,
            "dropout": dropout,
            "lr": lr,
            "batch_size": batch_size,
            "sequence_len": sequence_len,
            "MAE": np.mean(mae_scores),
            "RMSE": np.mean(rmse_scores),
            "MAPE": np.mean(mape_scores)
        })


🔧 Config 1/36: hidden=32, layers=1, dropout=0.0, lr=0.0005, batch_size=32, seq_len=30

🔧 Config 2/36: hidden=32, layers=1, dropout=0.1, lr=0.0005, batch_size=32, seq_len=30

🔧 Config 3/36: hidden=32, layers=1, dropout=0.2, lr=0.0005, batch_size=32, seq_len=30

🔧 Config 4/36: hidden=32, layers=1, dropout=0.3, lr=0.0005, batch_size=32, seq_len=30

🔧 Config 5/36: hidden=32, layers=2, dropout=0.0, lr=0.0005, batch_size=32, seq_len=30

🔧 Config 6/36: hidden=32, layers=2, dropout=0.1, lr=0.0005, batch_size=32, seq_len=30

🔧 Config 7/36: hidden=32, layers=2, dropout=0.2, lr=0.0005, batch_size=32, seq_len=30

🔧 Config 8/36: hidden=32, layers=2, dropout=0.3, lr=0.0005, batch_size=32, seq_len=30

🔧 Config 9/36: hidden=32, layers=3, dropout=0.0, lr=0.0005, batch_size=32, seq_len=30

🔧 Config 10/36: hidden=32, layers=3, dropout=0.1, lr=0.0005, batch_size=32, seq_len=30

🔧 Config 11/36: hidden=32, layers=3, dropout=0.2, lr=0.0005, batch_size=32, seq_len=30

🔧 Config 12/36: hidden=32, layers=3, dro

In [ ]:
df_results = pd.DataFrame(results)
df_results = df_results.sort_values("RMSE").reset_index(drop=True)
df_results

,hidden_size,num_layers,dropout,lr,batch_size,sequence_len,MAE,RMSE,MAPE
0,128,1,0.1,0.0005,32,30,31.989463,39.010419,93.621934
1,128,1,0.0,0.0005,32,30,32.030244,39.075393,93.287363
2,128,1,0.3,0.0005,32,30,32.161362,39.191993,94.412604
3,64,1,0.2,0.0005,32,30,32.201674,39.206890,93.123525
4,64,1,0.0,0.0005,32,30,32.433503,39.375493,93.828892
5,64,1,0.3,0.0005,32,30,32.473873,39.386544,92.980448
6,64,1,0.1,0.0005,32,30,32.484350,39.408041,93.480679
7,128,1,0.2,0.0005,32,30,32.428825,39.472570,95.636723
8,128,2,0.2,0.0005,32,30,32.625675,39.621058,94.560428
9,128,2,0.3,0.0005,32,30,32.781212,39.667261,93.740078


Hyperparameter tunning 3

In [ ]:
import itertools
import pandas as pd

# Define hyperparameter grid
hidden_sizes = [32, 64, 128]
num_layers_list = [1, 2, 3]
dropouts = [0.0, 0.1, 0.2, 0.3]
learning_rates = [0.001]
batch_sizes = [64]
sequence_lens = [30]

# Create grid of all combinations
param_grid = list(itertools.product(
    hidden_sizes,
    num_layers_list,
    dropouts,
    learning_rates,
    batch_sizes,
    sequence_lens
))

print(f"Total combinations: {len(param_grid)}")

🔍 Total combinations: 36


In [ ]:
results = []

for i, (hidden_size, num_layers, dropout, lr, batch_size, sequence_len) in enumerate(param_grid):
    print(f"\n Config {i+1}/{len(param_grid)}: hidden={hidden_size}, layers={num_layers}, dropout={dropout}, "
          f"lr={lr}, batch_size={batch_size}, seq_len={sequence_len}")

    mae_scores = []
    rmse_scores = []
    mape_scores = []

    for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
        expanding_window_cv(X, y,
                            initial_train_size=1095,
                            horizon=14,
                            step=14,
                            sequence_len=sequence_len)
    ):
        mae, rmse, mape = train_one_fold(
            X_tr, y_tr, X_val, y_val,
            sequence_len=sequence_len,
            num_epochs=50,
            patience=5,
            batch_size=batch_size,
            lr=lr,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout
        )

        if mae is None:
            continue  # skip invalid fold

        mae_scores.append(mae)
        rmse_scores.append(rmse)
        mape_scores.append(mape)

    if mae_scores:  # only record results if at least one fold succeeded
        results.append({
            "hidden_size": hidden_size,
            "num_layers": num_layers,
            "dropout": dropout,
            "lr": lr,
            "batch_size": batch_size,
            "sequence_len": sequence_len,
            "MAE": np.mean(mae_scores),
            "RMSE": np.mean(rmse_scores),
            "MAPE": np.mean(mape_scores)
        })


🔧 Config 1/36: hidden=32, layers=1, dropout=0.0, lr=0.001, batch_size=64, seq_len=30

🔧 Config 2/36: hidden=32, layers=1, dropout=0.1, lr=0.001, batch_size=64, seq_len=30

🔧 Config 3/36: hidden=32, layers=1, dropout=0.2, lr=0.001, batch_size=64, seq_len=30

🔧 Config 4/36: hidden=32, layers=1, dropout=0.3, lr=0.001, batch_size=64, seq_len=30

🔧 Config 5/36: hidden=32, layers=2, dropout=0.0, lr=0.001, batch_size=64, seq_len=30

🔧 Config 6/36: hidden=32, layers=2, dropout=0.1, lr=0.001, batch_size=64, seq_len=30

🔧 Config 7/36: hidden=32, layers=2, dropout=0.2, lr=0.001, batch_size=64, seq_len=30

🔧 Config 8/36: hidden=32, layers=2, dropout=0.3, lr=0.001, batch_size=64, seq_len=30

🔧 Config 9/36: hidden=32, layers=3, dropout=0.0, lr=0.001, batch_size=64, seq_len=30

🔧 Config 10/36: hidden=32, layers=3, dropout=0.1, lr=0.001, batch_size=64, seq_len=30

🔧 Config 11/36: hidden=32, layers=3, dropout=0.2, lr=0.001, batch_size=64, seq_len=30

🔧 Config 12/36: hidden=32, layers=3, dropout=0.3, l

In [ ]:
df_results = pd.DataFrame(results)
df_results = df_results.sort_values("RMSE").reset_index(drop=True)
df_results

,hidden_size,num_layers,dropout,lr,batch_size,sequence_len,MAE,RMSE,MAPE
0,64,1,0.1,0.001,64,30,31.740778,38.724843,90.856150
1,64,1,0.3,0.001,64,30,31.811634,38.816433,90.180240
2,64,1,0.2,0.001,64,30,31.803220,38.817081,89.708351
3,128,1,0.0,0.001,64,30,31.815912,38.843725,92.649494
4,128,1,0.2,0.001,64,30,31.888756,38.856672,93.157969
5,128,1,0.1,0.001,64,30,31.924930,38.976558,92.855464
6,64,1,0.0,0.001,64,30,32.027410,39.021642,91.803662
7,128,1,0.3,0.001,64,30,32.066919,39.081068,93.793686
8,32,1,0.0,0.001,64,30,32.522283,39.540283,93.827970
9,32,1,0.3,0.001,64,30,32.636247,39.672210,93.822617


Hyperparameter tunning 4

In [ ]:
import itertools
import pandas as pd

# Define hyperparameter grid
hidden_sizes = [32, 64, 128]
num_layers_list = [1, 2, 3]
dropouts = [0.0, 0.1, 0.2, 0.3]
learning_rates = [0.001]
batch_sizes = [64]
sequence_lens = [30]

# Create grid of all combinations
param_grid = list(itertools.product(
    hidden_sizes,
    num_layers_list,
    dropouts,
    learning_rates,
    batch_sizes,
    sequence_lens
))

print(f"Total combinations: {len(param_grid)}")

🔍 Total combinations: 36


In [ ]:
results = []

for i, (hidden_size, num_layers, dropout, lr, batch_size, sequence_len) in enumerate(param_grid):
    print(f"\n Config {i+1}/{len(param_grid)}: hidden={hidden_size}, layers={num_layers}, dropout={dropout}, "
          f"lr={lr}, batch_size={batch_size}, seq_len={sequence_len}")

    mae_scores = []
    rmse_scores = []
    mape_scores = []

    for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
        expanding_window_cv(X, y,
                            initial_train_size=1095,
                            horizon=14,
                            step=14,
                            sequence_len=sequence_len)
    ):
        mae, rmse, mape = train_one_fold(
            X_tr, y_tr, X_val, y_val,
            sequence_len=sequence_len,
            num_epochs=50,
            patience=5,
            batch_size=batch_size,
            lr=lr,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout
        )

        if mae is None:
            continue  # skip invalid fold

        mae_scores.append(mae)
        rmse_scores.append(rmse)
        mape_scores.append(mape)

    if mae_scores:  # only record results if at least one fold succeeded
        results.append({
            "hidden_size": hidden_size,
            "num_layers": num_layers,
            "dropout": dropout,
            "lr": lr,
            "batch_size": batch_size,
            "sequence_len": sequence_len,
            "MAE": np.mean(mae_scores),
            "RMSE": np.mean(rmse_scores),
            "MAPE": np.mean(mape_scores)
        })


🔧 Config 1/36: hidden=32, layers=1, dropout=0.0, lr=0.001, batch_size=64, seq_len=30

🔧 Config 2/36: hidden=32, layers=1, dropout=0.1, lr=0.001, batch_size=64, seq_len=30

🔧 Config 3/36: hidden=32, layers=1, dropout=0.2, lr=0.001, batch_size=64, seq_len=30

🔧 Config 4/36: hidden=32, layers=1, dropout=0.3, lr=0.001, batch_size=64, seq_len=30

🔧 Config 5/36: hidden=32, layers=2, dropout=0.0, lr=0.001, batch_size=64, seq_len=30

🔧 Config 6/36: hidden=32, layers=2, dropout=0.1, lr=0.001, batch_size=64, seq_len=30

🔧 Config 7/36: hidden=32, layers=2, dropout=0.2, lr=0.001, batch_size=64, seq_len=30

🔧 Config 8/36: hidden=32, layers=2, dropout=0.3, lr=0.001, batch_size=64, seq_len=30

🔧 Config 9/36: hidden=32, layers=3, dropout=0.0, lr=0.001, batch_size=64, seq_len=30

🔧 Config 10/36: hidden=32, layers=3, dropout=0.1, lr=0.001, batch_size=64, seq_len=30

🔧 Config 11/36: hidden=32, layers=3, dropout=0.2, lr=0.001, batch_size=64, seq_len=30

🔧 Config 12/36: hidden=32, layers=3, dropout=0.3, l

In [ ]:
df_results = pd.DataFrame(results)
df_results = df_results.sort_values("RMSE").reset_index(drop=True)
df_results

,hidden_size,num_layers,dropout,lr,batch_size,sequence_len,MAE,RMSE,MAPE
0,64,1,0.0,0.001,64,30,31.570565,38.519481,90.762250
1,64,1,0.2,0.001,64,30,31.723004,38.625461,91.024280
2,64,1,0.3,0.001,64,30,31.694365,38.689136,91.036445
3,128,1,0.1,0.001,64,30,31.846351,38.803713,91.054587
4,64,1,0.1,0.001,64,30,31.865811,38.832720,91.207396
5,128,1,0.0,0.001,64,30,31.890369,38.942444,91.766658
6,128,1,0.2,0.001,64,30,32.035692,39.084214,91.924754
7,128,1,0.3,0.001,64,30,32.142804,39.170698,90.108185
8,32,1,0.2,0.001,64,30,32.411718,39.388347,92.897095
9,32,1,0.1,0.001,64,30,32.702341,39.814171,94.447595


More restricted grid search

In [ ]:
import itertools
import pandas as pd
import numpy as np

# Restricted Grid
hidden_sizes = [64, 128]
num_layers_list = [1, 2]
lrs = [0.001, 0.0005]
dropouts = [0.1, 0.3]
sequence_lens = [30]

# Fixed parameters
batch_size = 64
num_epochs = 50
patience = 5
initial_train_size = 1095
horizon = 14
step = 14

# All combinations
param_grid = list(itertools.product(hidden_sizes, num_layers_list, lrs, dropouts, sequence_lens))

results = []

# Grid search loop
for i, (hidden_size, num_layers, lr, dropout, sequence_len) in enumerate(param_grid):
    print(f"\n Config {i+1}/{len(param_grid)} — hidden: {hidden_size}, layers: {num_layers}, "
          f"lr: {lr}, dropout: {dropout}, seq_len: {sequence_len}")

    mae_scores = []
    rmse_scores = []
    mape_scores = []

    for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
        expanding_window_cv(X, y, initial_train_size=initial_train_size, horizon=horizon, step=step, sequence_len=sequence_len)
    ):
        mae, rmse, mape = train_one_fold(
            X_tr, y_tr, X_val, y_val,
            sequence_len=sequence_len,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            lr=lr,
            batch_size=batch_size,
            num_epochs=num_epochs,
            patience=patience
        )
        if mae is not None:
            mae_scores.append(mae)
            rmse_scores.append(rmse)
            mape_scores.append(mape)

    # Aggregate fold results
    results.append({
        "hidden_size": hidden_size,
        "num_layers": num_layers,
        "lr": lr,
        "dropout": dropout,
        "sequence_len": sequence_len,
        "MAE": np.mean(mae_scores),
        "RMSE": np.mean(rmse_scores),
        "MAPE": np.mean(mape_scores)
    })

# Final DataFrame
df_lstm_results = pd.DataFrame(results)
df_lstm_results = df_lstm_results.sort_values("RMSE").reset_index(drop=True)
df_lstm_results


🔍 Config 1/16 — hidden: 64, layers: 1, lr: 0.001, dropout: 0.1, seq_len: 30

🔍 Config 2/16 — hidden: 64, layers: 1, lr: 0.001, dropout: 0.3, seq_len: 30

🔍 Config 3/16 — hidden: 64, layers: 1, lr: 0.0005, dropout: 0.1, seq_len: 30

🔍 Config 4/16 — hidden: 64, layers: 1, lr: 0.0005, dropout: 0.3, seq_len: 30

🔍 Config 5/16 — hidden: 64, layers: 2, lr: 0.001, dropout: 0.1, seq_len: 30

🔍 Config 6/16 — hidden: 64, layers: 2, lr: 0.001, dropout: 0.3, seq_len: 30

🔍 Config 7/16 — hidden: 64, layers: 2, lr: 0.0005, dropout: 0.1, seq_len: 30

🔍 Config 8/16 — hidden: 64, layers: 2, lr: 0.0005, dropout: 0.3, seq_len: 30

🔍 Config 9/16 — hidden: 128, layers: 1, lr: 0.001, dropout: 0.1, seq_len: 30

🔍 Config 10/16 — hidden: 128, layers: 1, lr: 0.001, dropout: 0.3, seq_len: 30

🔍 Config 11/16 — hidden: 128, layers: 1, lr: 0.0005, dropout: 0.1, seq_len: 30

🔍 Config 12/16 — hidden: 128, layers: 1, lr: 0.0005, dropout: 0.3, seq_len: 30

🔍 Config 13/16 — hidden: 128, layers: 2, lr: 0.001, dropout: 0

,hidden_size,num_layers,lr,dropout,sequence_len,MAE,RMSE,MAPE
0,128,1,0.0005,0.1,30,31.597565,38.555926,90.170319
1,128,1,0.0010,0.1,30,31.627081,38.561881,92.077852
2,64,1,0.0010,0.1,30,31.612207,38.619039,89.960110
3,64,1,0.0010,0.3,30,31.668869,38.688395,92.812037
4,128,1,0.0005,0.3,30,31.709690,38.689317,91.470480
5,128,1,0.0010,0.3,30,32.159439,39.110712,94.202051
6,64,1,0.0005,0.1,30,32.484008,39.427880,94.260430
7,64,1,0.0005,0.3,30,32.692376,39.714518,94.046512
8,64,2,0.0005,0.1,30,32.996730,40.008485,94.827266
9,128,2,0.0010,0.3,30,33.184928,40.073987,91.779999


GRU

In [ ]:
class GRUForecastNet(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=1, dropout=0.2, output_size=14):
        super().__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers,
                          batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        gru_out, _ = self.gru(x)  # [batch, seq, hidden]
        out = self.fc(gru_out[:, -1, :])  # [batch, output_size]
        return out

In [ ]:
def train_one_gru_fold(X_train, y_train, X_val, y_val,
                       sequence_len=30, num_epochs=50, patience=5,
                       batch_size=32, lr=0.001, hidden_size=64, num_layers=1, dropout=0.2):

    # --- 1. Scaling ---
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train_scaled = scaler_X.fit_transform(X_train)
    X_val_scaled = scaler_X.transform(X_val)

    y_train_scaled = scaler_y.fit_transform(y_train)  # shape: (n_samples, 14)
    y_val_scaled = scaler_y.transform(y_val)

    # --- 2. Create sequences ---
    X_train_seq, y_train_seq = create_lstm_input(X_train_scaled, y_train_scaled, sequence_len)
    X_val_seq, y_val_seq = create_lstm_input(X_val_scaled, y_val_scaled, sequence_len)

    if X_val_seq.shape[0] == 0:
        print("Skipping fold: Not enough validation sequences.")
        return None, None, None

    # --- 3. Data loaders ---
    train_loader = DataLoader(
        LSTMForecastDataset(X_train_seq, y_train_seq), batch_size=batch_size, shuffle=False
    )
    val_loader = DataLoader(
        LSTMForecastDataset(X_val_seq, y_val_seq), batch_size=batch_size, shuffle=False
    )

    # --- 4. Model setup ---
    model = GRUForecastNet(
        input_size=X_train_seq.shape[2],
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout=dropout
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None

    # --- 5. Training loop ---
    for epoch in range(num_epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)

            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            val_preds = torch.cat([model(xb.to(device)) for xb, _ in val_loader])
            val_targets = torch.cat([yb.to(device) for _, yb in val_loader])
            val_loss = criterion(val_preds, val_targets)

        if val_loss.item() < best_val_loss - 1e-4:
            best_val_loss = val_loss.item()
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break

    # --- 6. Load best model ---
    if best_model_state:
        model.load_state_dict(best_model_state)

    # --- 7. Final evaluation ---
    model.eval()
    with torch.no_grad():
        X_val_tensor = torch.tensor(X_val_seq, dtype=torch.float32).to(device)
        preds_scaled = model(X_val_tensor).cpu().numpy()
        targets_scaled = y_val_seq

    preds = scaler_y.inverse_transform(preds_scaled)
    targets = scaler_y.inverse_transform(targets_scaled)

    # --- 8. Metrics ---
    mae = mean_absolute_error(targets.flatten(), preds.flatten())
    rmse = np.sqrt(mean_squared_error(targets.flatten(), preds.flatten()))
    mape = mean_absolute_percentage_error(targets.flatten(), preds.flatten()) * 100

    return mae, rmse, mape

In [ ]:
sequence_len=30

mae_scores = []
rmse_scores = []
mape_scores = []

for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
    expanding_window_cv(X, y, initial_train_size=1095, horizon=14, step=14, sequence_len=sequence_len)
):
    print(f"\nFold {fold+1}")
    mae, rmse, mape = train_one_gru_fold(
        X_tr, y_tr, X_val, y_val, sequence_len=sequence_len
    )
    if mae is None:
        continue  # Skip if fold was invalid
    print(f"  MAE: {mae:.4f}, RMSE: {rmse:.4f}, MAPE: {mape:.2f}")
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    mape_scores.append(mape)

# --- Final CV Results ---
print("\n Final CV Results:")
print(f"Average MAE:  {np.mean(mae_scores):.4f}")
print(f"Average RMSE: {np.mean(rmse_scores):.4f}")
print(f"Average MAPE: {np.mean(mape_scores):.4f}")


Fold 1
  MAE: 9.6373, RMSE: 11.2689, MAPE: 36.96

Fold 2
  MAE: 9.2273, RMSE: 11.0179, MAPE: 58.69

Fold 3
  MAE: 7.5274, RMSE: 9.1175, MAPE: 47.51

Fold 4
  MAE: 5.1510, RMSE: 6.2126, MAPE: 14.64

Fold 5
  MAE: 5.5311, RMSE: 7.0806, MAPE: 19.67

Fold 6
  MAE: 6.6350, RMSE: 8.2622, MAPE: 24.63

Fold 7
  MAE: 9.1826, RMSE: 10.2782, MAPE: 24.70

Fold 8
  MAE: 9.9009, RMSE: 10.6315, MAPE: 22.38

Fold 9
  MAE: 11.3089, RMSE: 12.2748, MAPE: 24.45

Fold 10
  MAE: 14.9825, RMSE: 16.4640, MAPE: 30.42

Fold 11
  MAE: 14.5377, RMSE: 15.1059, MAPE: 27.21

Fold 12
  MAE: 11.9211, RMSE: 13.0414, MAPE: 21.44

Fold 13
  MAE: 11.3326, RMSE: 12.6223, MAPE: 19.66

Fold 14
  MAE: 11.8640, RMSE: 13.4460, MAPE: 19.75

Fold 15
  MAE: 11.6782, RMSE: 13.7131, MAPE: 25.11

Fold 16
  MAE: 11.3090, RMSE: 13.7987, MAPE: 32.41

Fold 17
  MAE: 12.5166, RMSE: 15.0245, MAPE: 33.31

Fold 18
  MAE: 10.1741, RMSE: 12.6082, MAPE: 26.39

Fold 19
  MAE: 9.5353, RMSE: 12.2500, MAPE: 18.04

Fold 20
  MAE: 11.4197, RMSE: 15.

Hyperparamter tunning

In [ ]:
import itertools

# Hyperparameter grid for GRU
hidden_sizes = [32, 64, 128]
num_layers_list = [1, 2, 3]
dropouts = [0.0, 0.1, 0.2, 0.3]
learning_rates = [0.001, 0.0005]
batch_sizes = [32, 64]

# Cartesian product of all combinations
gru_param_grid = list(itertools.product(hidden_sizes, num_layers_list, dropouts, learning_rates, batch_sizes))
print(f"Total GRU configurations: {len(gru_param_grid)}")

Total GRU configurations: 144


In [ ]:
gru_results = []

for i, (hidden_size, num_layers, dropout, lr, batch_size) in enumerate(gru_param_grid):
    print(f"\n Config {i+1}/{len(gru_param_grid)} → hidden={hidden_size}, layers={num_layers}, dropout={dropout}, lr={lr}, batch_size={batch_size}")

    mae_scores = []
    rmse_scores = []
    mape_scores = []

    for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
        expanding_window_cv(X, y, initial_train_size=1095, horizon=14, step=14, sequence_len=30)
    ):
        mae, rmse, mape = train_one_gru_fold(
            X_tr, y_tr, X_val, y_val,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            lr=lr,
            batch_size=batch_size,
            num_epochs=50,
            patience=5,
            sequence_len=30
        )

        if mae is None:
            continue
        mae_scores.append(mae)
        rmse_scores.append(rmse)
        mape_scores.append(mape)

    # Save results
    avg_mae = np.mean(mae_scores)
    avg_rmse = np.mean(rmse_scores)
    avg_mape = np.mean(mape_scores)

    gru_results.append({
        "hidden_size": hidden_size,
        "num_layers": num_layers,
        "dropout": dropout,
        "lr": lr,
        "batch_size": batch_size,
        "MAE": avg_mae,
        "RMSE": avg_rmse,
        "MAPE": avg_mape
    })

    print(f"Avg MAE: {avg_mae:.4f} | RMSE: {avg_rmse:.4f} | MAPE: {avg_mape:.4f}")


🔧 Config 1/144 → hidden=32, layers=1, dropout=0.0, lr=0.001, batch_size=32
Avg MAE: 31.0825 | RMSE: 37.8943 | MAPE: 87.36

🔧 Config 2/144 → hidden=32, layers=1, dropout=0.0, lr=0.001, batch_size=64
Avg MAE: 31.4265 | RMSE: 38.2849 | MAPE: 91.81

🔧 Config 3/144 → hidden=32, layers=1, dropout=0.0, lr=0.0005, batch_size=32
Avg MAE: 31.8264 | RMSE: 38.7084 | MAPE: 91.77

🔧 Config 4/144 → hidden=32, layers=1, dropout=0.0, lr=0.0005, batch_size=64
Avg MAE: 32.0994 | RMSE: 39.0682 | MAPE: 94.95

🔧 Config 5/144 → hidden=32, layers=1, dropout=0.1, lr=0.001, batch_size=32
Avg MAE: 31.2114 | RMSE: 37.9457 | MAPE: 85.04

🔧 Config 6/144 → hidden=32, layers=1, dropout=0.1, lr=0.001, batch_size=64
Avg MAE: 31.4460 | RMSE: 38.2650 | MAPE: 90.28

🔧 Config 7/144 → hidden=32, layers=1, dropout=0.1, lr=0.0005, batch_size=32
Avg MAE: 31.8464 | RMSE: 38.7196 | MAPE: 92.42

🔧 Config 8/144 → hidden=32, layers=1, dropout=0.1, lr=0.0005, batch_size=64
Avg MAE: 32.0268 | RMSE: 38.9262 | MAPE: 94.39

🔧 Config 9/

In [ ]:
import pandas as pd

df_gru_results = pd.DataFrame(gru_results)
df_gru_results = df_gru_results.sort_values(by="RMSE").reset_index(drop=True)
df_gru_results.head()

NameError: name 'gru_results' is not defined

More restrcited hyperparamter search (hidden size 32)

In [ ]:
import itertools

# Hyperparameter grid for GRU
hidden_sizes = [32]
num_layers_list = [1, 2, 3]
dropouts = [0.0, 0.1, 0.2, 0.3]
learning_rates = [0.001, 0.0005]
batch_sizes = [32]

# Cartesian product of all combinations
gru_param_grid = list(itertools.product(hidden_sizes, num_layers_list, dropouts, learning_rates, batch_sizes))
print(f"Total GRU configurations: {len(gru_param_grid)}")

Total GRU configurations: 24


In [ ]:
gru_results = []

for i, (hidden_size, num_layers, dropout, lr, batch_size) in enumerate(gru_param_grid):
    print(f"\n Config {i+1}/{len(gru_param_grid)} → hidden={hidden_size}, layers={num_layers}, dropout={dropout}, lr={lr}, batch_size={batch_size}")

    mae_scores = []
    rmse_scores = []
    mape_scores = []

    for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
        expanding_window_cv(X, y, initial_train_size=1095, horizon=14, step=14, sequence_len=30)
    ):
        mae, rmse, mape = train_one_gru_fold(
            X_tr, y_tr, X_val, y_val,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            lr=lr,
            batch_size=batch_size,
            num_epochs=50,
            patience=5,
            sequence_len=30
        )

        if mae is None:
            continue
        mae_scores.append(mae)
        rmse_scores.append(rmse)
        mape_scores.append(mape)

    # Save results
    avg_mae = np.mean(mae_scores)
    avg_rmse = np.mean(rmse_scores)
    avg_mape = np.mean(mape_scores)

    gru_results.append({
        "hidden_size": hidden_size,
        "num_layers": num_layers,
        "dropout": dropout,
        "lr": lr,
        "batch_size": batch_size,
        "MAE": avg_mae,
        "RMSE": avg_rmse,
        "MAPE": avg_mape
    })

    print(f"Avg MAE: {avg_mae:.4f} | RMSE: {avg_rmse:.4f} | MAPE: {avg_mape:.4f}")


🔧 Config 1/24 → hidden=32, layers=1, dropout=0.0, lr=0.001, batch_size=32
Avg MAE: 31.0175 | RMSE: 37.8457 | MAPE: 88.0613

🔧 Config 2/24 → hidden=32, layers=1, dropout=0.0, lr=0.0005, batch_size=32
Avg MAE: 32.0763 | RMSE: 38.9613 | MAPE: 92.3032

🔧 Config 3/24 → hidden=32, layers=1, dropout=0.1, lr=0.001, batch_size=32


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers

Avg MAE: 31.6715 | RMSE: 38.5261 | MAPE: 88.2952

🔧 Config 4/24 → hidden=32, layers=1, dropout=0.1, lr=0.0005, batch_size=32


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers

Avg MAE: 31.8365 | RMSE: 38.6667 | MAPE: 92.1337

🔧 Config 5/24 → hidden=32, layers=1, dropout=0.2, lr=0.001, batch_size=32


...

Avg MAE: 31.7324 | RMSE: 38.6063 | MAPE: 91.9527

🔧 Config 7/24 → hidden=32, layers=1, dropout=0.3, lr=0.001, batch_size=32


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers

Avg MAE: 31.1420 | RMSE: 37.9579 | MAPE: 87.8831

🔧 Config 8/24 → hidden=32, layers=1, dropout=0.3, lr=0.0005, batch_size=32


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers

Avg MAE: 31.8329 | RMSE: 38.6968 | MAPE: 92.1387

🔧 Config 9/24 → hidden=32, layers=2, dropout=0.0, lr=0.001, batch_size=32
Avg MAE: 32.0532 | RMSE: 38.9817 | MAPE: 88.8110

🔧 Config 10/24 → hidden=32, layers=2, dropout=0.0, lr=0.0005, batch_size=32
Avg MAE: 32.1210 | RMSE: 38.9676 | MAPE: 93.1882

🔧 Config 11/24 → hidden=32, layers=2, dropout=0.1, lr=0.001, batch_size=32
Avg MAE: 31.6814 | RMSE: 38.5619 | MAPE: 90.9705

🔧 Config 12/24 → hidden=32, layers=2, dropout=0.1, lr=0.0005, batch_size=32
Avg MAE: 32.2432 | RMSE: 39.0620 | MAPE: 92.7419

🔧 Config 13/24 → hidden=32, layers=2, dropout=0.2, lr=0.001, batch_size=32
Avg MAE: 31.8684 | RMSE: 38.6939 | MAPE: 91.4164

🔧 Config 14/24 → hidden=32, layers=2, dropout=0.2, lr=0.0005, batch_size=32
Avg MAE: 32.5504 | RMSE: 39.4341 | MAPE: 93.3800

🔧 Config 15/24 → hidden=32, layers=2, dropout=0.3, lr=0.001, batch_size=32
Avg MAE: 31.9075 | RMSE: 38.7952 | MAPE: 88.9838

🔧 Config 16/24 → hidden=32, layers=2, dropout=0.3, lr=0.0005, batch_size=

In [ ]:
import pandas as pd

df_gru_results = pd.DataFrame(gru_results)
df_gru_results = df_gru_results.sort_values(by="RMSE").reset_index(drop=True)
df_gru_results

,hidden_size,num_layers,dropout,lr,batch_size,MAE,RMSE,MAPE
0,32,1,0.0,0.0010,32,31.017536,37.845745,88.061268
1,32,1,0.3,0.0010,32,31.141957,37.957894,87.883125
2,32,1,0.2,0.0010,32,31.344381,38.161029,89.392035
3,32,1,0.1,0.0010,32,31.671511,38.526077,88.295192
4,32,2,0.1,0.0010,32,31.681379,38.561885,90.970484
5,32,1,0.2,0.0005,32,31.732448,38.606335,91.952732
6,32,1,0.1,0.0005,32,31.836496,38.666732,92.133740
7,32,2,0.2,0.0010,32,31.868374,38.693949,91.416412
8,32,1,0.3,0.0005,32,31.832950,38.696790,92.138664
9,32,2,0.3,0.0010,32,31.907455,38.795204,88.983806


Hidden size = 64

In [ ]:
import itertools

# Hyperparameter grid for GRU
hidden_sizes = [64]
num_layers_list = [1, 2, 3]
dropouts = [0.0, 0.1, 0.2, 0.3]
learning_rates = [0.001, 0.0005]
batch_sizes = [32]

# Cartesian product of all combinations
gru_param_grid = list(itertools.product(hidden_sizes, num_layers_list, dropouts, learning_rates, batch_sizes))
print(f"Total GRU configurations: {len(gru_param_grid)}")

Total GRU configurations: 24


In [ ]:
gru_results = []

for i, (hidden_size, num_layers, dropout, lr, batch_size) in enumerate(gru_param_grid):
    print(f"\n Config {i+1}/{len(gru_param_grid)} → hidden={hidden_size}, layers={num_layers}, dropout={dropout}, lr={lr}, batch_size={batch_size}")

    mae_scores = []
    rmse_scores = []
    mape_scores = []

    for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
        expanding_window_cv(X, y, initial_train_size=1095, horizon=14, step=14, sequence_len=30)
    ):
        mae, rmse, mape = train_one_gru_fold(
            X_tr, y_tr, X_val, y_val,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            lr=lr,
            batch_size=batch_size,
            num_epochs=50,
            patience=5,
            sequence_len=30
        )

        if mae is None:
            continue
        mae_scores.append(mae)
        rmse_scores.append(rmse)
        mape_scores.append(mape)

    # Save results
    avg_mae = np.mean(mae_scores)
    avg_rmse = np.mean(rmse_scores)
    avg_mape = np.mean(mape_scores)

    gru_results.append({
        "hidden_size": hidden_size,
        "num_layers": num_layers,
        "dropout": dropout,
        "lr": lr,
        "batch_size": batch_size,
        "MAE": avg_mae,
        "RMSE": avg_rmse,
        "MAPE": avg_mape
    })

    print(f"Avg MAE: {avg_mae:.4f} | RMSE: {avg_rmse:.4f} | MAPE: {avg_mape:.4f}")


🔧 Config 1/24 → hidden=64, layers=1, dropout=0.0, lr=0.001, batch_size=32
Avg MAE: 30.8349 | RMSE: 37.7222 | MAPE: 87.8089

🔧 Config 2/24 → hidden=64, layers=1, dropout=0.0, lr=0.0005, batch_size=32
Avg MAE: 30.8407 | RMSE: 37.6596 | MAPE: 88.4100

🔧 Config 3/24 → hidden=64, layers=1, dropout=0.1, lr=0.001, batch_size=32
Avg MAE: 30.7921 | RMSE: 37.6373 | MAPE: 88.2749

🔧 Config 4/24 → hidden=64, layers=1, dropout=0.1, lr=0.0005, batch_size=32
Avg MAE: 30.9867 | RMSE: 37.8077 | MAPE: 88.2092

🔧 Config 5/24 → hidden=64, layers=1, dropout=0.2, lr=0.001, batch_size=32
Avg MAE: 30.9566 | RMSE: 37.8194 | MAPE: 87.3977

🔧 Config 6/24 → hidden=64, layers=1, dropout=0.2, lr=0.0005, batch_size=32
Avg MAE: 30.9272 | RMSE: 37.7386 | MAPE: 88.5315

🔧 Config 7/24 → hidden=64, layers=1, dropout=0.3, lr=0.001, batch_size=32
Avg MAE: 30.6823 | RMSE: 37.5069 | MAPE: 86.5780

🔧 Config 8/24 → hidden=64, layers=1, dropout=0.3, lr=0.0005, batch_size=32
Avg MAE: 30.8600 | RMSE: 37.6720 | MAPE: 89.0100

🔧 C

In [ ]:
import pandas as pd

df_gru_results = pd.DataFrame(gru_results)
df_gru_results = df_gru_results.sort_values(by="RMSE").reset_index(drop=True)
df_gru_results

,hidden_size,num_layers,dropout,lr,batch_size,MAE,RMSE,MAPE
0,64,1,0.3,0.0010,32,30.682320,37.506927,86.578029
1,64,1,0.1,0.0010,32,30.792061,37.637256,88.274853
2,64,1,0.0,0.0005,32,30.840682,37.659643,88.409994
3,64,1,0.3,0.0005,32,30.859953,37.671996,89.009969
4,64,1,0.0,0.0010,32,30.834857,37.722187,87.808941
5,64,1,0.2,0.0005,32,30.927238,37.738610,88.531531
6,64,1,0.1,0.0005,32,30.986737,37.807722,88.209191
7,64,1,0.2,0.0010,32,30.956565,37.819388,87.397746
8,64,2,0.3,0.0005,32,31.362332,38.143227,88.533290
9,64,2,0.1,0.0005,32,31.408799,38.198385,88.458445


hidden size = 128

In [ ]:
import itertools

# Hyperparameter grid for GRU
hidden_sizes = [128]
num_layers_list = [1, 2, 3]
dropouts = [0.0, 0.1, 0.2, 0.3]
learning_rates = [0.001, 0.0005]
batch_sizes = [32]

# Cartesian product of all combinations
gru_param_grid = list(itertools.product(hidden_sizes, num_layers_list, dropouts, learning_rates, batch_sizes))
print(f"Total GRU configurations: {len(gru_param_grid)}")

Total GRU configurations: 24


In [ ]:
gru_results = []

for i, (hidden_size, num_layers, dropout, lr, batch_size) in enumerate(gru_param_grid):
    print(f"\n Config {i+1}/{len(gru_param_grid)} → hidden={hidden_size}, layers={num_layers}, dropout={dropout}, lr={lr}, batch_size={batch_size}")

    mae_scores = []
    rmse_scores = []
    mape_scores = []

    for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
        expanding_window_cv(X, y, initial_train_size=1095, horizon=14, step=14, sequence_len=30)
    ):
        mae, rmse, mape = train_one_gru_fold(
            X_tr, y_tr, X_val, y_val,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            lr=lr,
            batch_size=batch_size,
            num_epochs=50,
            patience=5,
            sequence_len=30
        )

        if mae is None:
            continue
        mae_scores.append(mae)
        rmse_scores.append(rmse)
        mape_scores.append(mape)

    # Save results
    avg_mae = np.mean(mae_scores)
    avg_rmse = np.mean(rmse_scores)
    avg_mape = np.mean(mape_scores)

    gru_results.append({
        "hidden_size": hidden_size,
        "num_layers": num_layers,
        "dropout": dropout,
        "lr": lr,
        "batch_size": batch_size,
        "MAE": avg_mae,
        "RMSE": avg_rmse,
        "MAPE": avg_mape
    })

    print(f"Avg MAE: {avg_mae:.4f} | RMSE: {avg_rmse:.4f} | MAPE: {avg_mape:.4f}")


🔧 Config 1/24 → hidden=128, layers=1, dropout=0.0, lr=0.001, batch_size=32
Avg MAE: 31.2240 | RMSE: 38.2978 | MAPE: 88.8398

🔧 Config 2/24 → hidden=128, layers=1, dropout=0.0, lr=0.0005, batch_size=32
Avg MAE: 30.6073 | RMSE: 37.4164 | MAPE: 87.0896

🔧 Config 3/24 → hidden=128, layers=1, dropout=0.1, lr=0.001, batch_size=32
Avg MAE: 30.3729 | RMSE: 37.2731 | MAPE: 87.6090

🔧 Config 4/24 → hidden=128, layers=1, dropout=0.1, lr=0.0005, batch_size=32
Avg MAE: 30.4489 | RMSE: 37.3187 | MAPE: 86.9825

🔧 Config 5/24 → hidden=128, layers=1, dropout=0.2, lr=0.001, batch_size=32
Avg MAE: 31.4025 | RMSE: 38.4069 | MAPE: 89.0326

🔧 Config 6/24 → hidden=128, layers=1, dropout=0.2, lr=0.0005, batch_size=32
Avg MAE: 30.5978 | RMSE: 37.4325 | MAPE: 85.9272

🔧 Config 7/24 → hidden=128, layers=1, dropout=0.3, lr=0.001, batch_size=32
Avg MAE: 31.4151 | RMSE: 38.3928 | MAPE: 91.2921

🔧 Config 8/24 → hidden=128, layers=1, dropout=0.3, lr=0.0005, batch_size=32
Avg MAE: 30.7724 | RMSE: 37.5631 | MAPE: 88.7

In [ ]:
import pandas as pd

df_gru_results = pd.DataFrame(gru_results)
df_gru_results = df_gru_results.sort_values(by="RMSE").reset_index(drop=True)
df_gru_results

,hidden_size,num_layers,dropout,lr,batch_size,MAE,RMSE,MAPE
0,128,1,0.1,0.0010,32,30.372874,37.273085,87.608985
1,128,1,0.1,0.0005,32,30.448919,37.318707,86.982484
2,128,1,0.0,0.0005,32,30.607306,37.416392,87.089638
3,128,1,0.2,0.0005,32,30.597763,37.432491,85.927161
4,128,1,0.3,0.0005,32,30.772374,37.563075,88.751821
5,128,2,0.1,0.0005,32,30.884410,37.681634,87.047427
6,128,2,0.0,0.0005,32,31.119612,37.978445,88.058530
7,128,2,0.2,0.0005,32,31.448987,38.179006,88.106218
8,128,2,0.3,0.0005,32,31.329255,38.290526,90.033279
9,128,1,0.0,0.0010,32,31.224048,38.297774,88.839790


1D CNN

In [ ]:
import torch
import torch.nn as nn

class CNN1DForecastNet(nn.Module):
    def __init__(self, input_channels, seq_len, num_outputs=14, num_filters=64, kernel_size=3, dropout=0.2):
        super().__init__()

        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=num_filters, kernel_size=kernel_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

        # Compute output length after conv: L_out = L_in - kernel_size + 1
        conv_out_len = seq_len - kernel_size + 1

        self.flatten_dim = num_filters * conv_out_len
        self.fc = nn.Linear(self.flatten_dim, num_outputs)

    def forward(self, x):
        # x: [batch_size, seq_len, input_channels]
        x = x.permute(0, 2, 1)  # to shape [batch_size, input_channels, seq_len]
        x = self.conv1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)  # flatten
        x = self.fc(x)
        return x

In [ ]:
def train_one_cnn_fold(X_train, y_train, X_val, y_val,
                       sequence_len=30,
                       num_outputs=14,
                       num_filters=64,
                       kernel_size=3,
                       dropout=0.2,
                       num_epochs=50,
                       patience=5,
                       batch_size=32,
                       lr=0.001):

    # --- Scaling ---
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train_scaled = scaler_X.fit_transform(X_train)
    X_val_scaled = scaler_X.transform(X_val)

    y_train_scaled = scaler_y.fit_transform(y_train)
    y_val_scaled = scaler_y.transform(y_val)

    # --- Sequence creation ---
    X_train_seq, y_train_seq = create_lstm_input(X_train_scaled, y_train_scaled, sequence_len)
    X_val_seq, y_val_seq = create_lstm_input(X_val_scaled, y_val_scaled, sequence_len)

    if X_val_seq.shape[0] == 0:
        print("Skipping fold: Not enough validation sequences.")
        return None, None, None

    # --- DataLoader ---
    train_loader = DataLoader(LSTMForecastDataset(X_train_seq, y_train_seq), batch_size=batch_size, shuffle=False)
    val_loader = DataLoader(LSTMForecastDataset(X_val_seq, y_val_seq), batch_size=batch_size, shuffle=False)

    # --- Model ---
    model = CNN1DForecastNet(
        input_channels=X_train_seq.shape[2],
        seq_len=sequence_len,
        num_outputs=num_outputs,
        num_filters=num_filters,
        kernel_size=kernel_size,
        dropout=dropout
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    best_val_loss = float('inf')
    best_model_state = None
    patience_counter = 0

    # --- Training ---
    for epoch in range(num_epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()

        # --- Validation ---
        model.eval()
        with torch.no_grad():
            val_preds = torch.cat([model(xb.to(device)) for xb, _ in val_loader])
            val_targets = torch.cat([yb.to(device) for _, yb in val_loader])
            val_loss = criterion(val_preds, val_targets)

        if val_loss.item() < best_val_loss - 1e-4:
            best_val_loss = val_loss.item()
            best_model_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break

    # --- Load best model ---
    model.load_state_dict(best_model_state)

    # --- Final Evaluation ---
    model.eval()
    with torch.no_grad():
        X_val_tensor = torch.tensor(X_val_seq, dtype=torch.float32).to(device)
        preds_scaled = model(X_val_tensor).cpu().numpy()

    preds = scaler_y.inverse_transform(preds_scaled)
    targets = scaler_y.inverse_transform(y_val_seq)

    mae = mean_absolute_error(targets.flatten(), preds.flatten())
    rmse = np.sqrt(mean_squared_error(targets.flatten(), preds.flatten()))
    mape = mean_absolute_percentage_error(targets.flatten(), preds.flatten()) * 100

    return mae, rmse, mape

In [ ]:
sequence_len = 30
horizon = 14
initial_train_size = 1095

mae_scores = []
rmse_scores = []
mape_scores = []

for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
    expanding_window_cv(X, y, initial_train_size=initial_train_size, horizon=horizon, step=horizon, sequence_len=sequence_len)
):
    print(f"\nFold {fold+1}")
    mae, rmse, mape = train_one_cnn_fold(
        X_tr, y_tr, X_val, y_val,
        sequence_len=sequence_len,
        num_epochs=50,
        patience=5,
        batch_size=32,
        lr=0.001,
        num_filters=64,
        kernel_size=3,
        dropout=0.2
    )

    if mae is None:
        continue  # skip invalid fold

    print(f"  MAE: {mae:.4f} | RMSE: {rmse:.4f} | MAPE: {mape:.4f}")
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    mape_scores.append(mape)

# --- 7. Final CV Results ---
print("\n Final CNN CV Results:")
print(f"Average MAE:  {np.mean(mae_scores):.4f}")
print(f"Average RMSE: {np.mean(rmse_scores):.4f}")
print(f"Average MAPE: {np.mean(mape_scores):.4f}")


Fold 1
  MAE: 9.2065 | RMSE: 10.9951 | MAPE: 37.3955

Fold 2
  MAE: 8.1604 | RMSE: 10.4018 | MAPE: 60.6735

Fold 3
  MAE: 6.2092 | RMSE: 8.4651 | MAPE: 47.6519

Fold 4
  MAE: 4.1062 | RMSE: 5.2466 | MAPE: 12.1599

Fold 5
  MAE: 5.6599 | RMSE: 7.6750 | MAPE: 21.4291

Fold 6
  MAE: 6.3010 | RMSE: 8.3912 | MAPE: 25.3544

Fold 7
  MAE: 7.4965 | RMSE: 8.7249 | MAPE: 21.7351

Fold 8
  MAE: 7.8186 | RMSE: 8.7940 | MAPE: 18.4438

Fold 9
  MAE: 8.5261 | RMSE: 9.6369 | MAPE: 19.4468

Fold 10
  MAE: 4.7684 | RMSE: 6.9899 | MAPE: 11.5839

Fold 11
  MAE: 2.3379 | RMSE: 3.1121 | MAPE: 4.4672

Fold 12
  MAE: 6.2168 | RMSE: 8.0020 | MAPE: 12.0485

Fold 13
  MAE: 5.0222 | RMSE: 6.4630 | MAPE: 9.6629

Fold 14
  MAE: 5.8223 | RMSE: 7.6754 | MAPE: 11.0123

Fold 15
  MAE: 11.8153 | RMSE: 15.6474 | MAPE: 31.2153

Fold 16
  MAE: 16.0109 | RMSE: 19.5513 | MAPE: 48.2376

Fold 17
  MAE: 12.8769 | RMSE: 15.7129 | MAPE: 34.9465

Fold 18
  MAE: 9.2976 | RMSE: 11.6386 | MAPE: 23.7148

Fold 19
  MAE: 9.2851 | RMSE:

Initial 1-layer hyperparameter tunning

In [ ]:
import itertools

num_filters_list = [32, 64, 128]
kernel_sizes = [2, 3, 5]
dropouts = [0.0, 0.1, 0.2]
learning_rates = [0.001]
batch_sizes = [32]

cnn_grid = list(itertools.product(num_filters_list, kernel_sizes, dropouts, learning_rates, batch_sizes))
print(f"Total CNN configurations: {len(cnn_grid)}")

🔧 Total CNN configurations: 27


In [ ]:
cnn_results = []

for i, (num_filters, kernel_size, dropout, lr, batch_size) in enumerate(cnn_grid):
    print(f"\n Config {i+1}/{len(cnn_grid)} → filters={num_filters}, kernel={kernel_size}, dropout={dropout}, lr={lr}, batch={batch_size}")

    mae_scores = []
    rmse_scores = []
    mape_scores = []

    for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
        expanding_window_cv(X, y, initial_train_size=1095, horizon=14, step=14, sequence_len=30)
    ):
        mae, rmse, mape = train_one_cnn_fold(
            X_tr, y_tr, X_val, y_val,
            sequence_len=30,
            num_epochs=50,
            patience=5,
            batch_size=batch_size,
            lr=lr,
            num_filters=num_filters,
            kernel_size=kernel_size,
            dropout=dropout
        )

        if mae is not None:
            mae_scores.append(mae)
            rmse_scores.append(rmse)
            mape_scores.append(mape)

    if mae_scores:
        cnn_results.append({
            "filters": num_filters,
            "kernel": kernel_size,
            "dropout": dropout,
            "lr": lr,
            "batch": batch_size,
            "MAE": np.mean(mae_scores),
            "RMSE": np.mean(rmse_scores),
            "MAPE": np.mean(mape_scores)
        })

        print(f"Avg MAE: {np.mean(mae_scores):.2f} | RMSE: {np.mean(rmse_scores):.2f} | MAPE: {np.mean(mape_scores):.2f}")
    else:
        print("Skipped config due to missing fold results")


🔧 Config 1/27 → filters=32, kernel=2, dropout=0.0, lr=0.001, batch=32
✅ Avg MAE: 35.10 | RMSE: 42.50 | MAPE: 109.17

🔧 Config 2/27 → filters=32, kernel=2, dropout=0.1, lr=0.001, batch=32
✅ Avg MAE: 33.66 | RMSE: 40.99 | MAPE: 103.34

🔧 Config 3/27 → filters=32, kernel=2, dropout=0.2, lr=0.001, batch=32
✅ Avg MAE: 34.17 | RMSE: 41.41 | MAPE: 106.29

🔧 Config 4/27 → filters=32, kernel=3, dropout=0.0, lr=0.001, batch=32
✅ Avg MAE: 34.11 | RMSE: 41.54 | MAPE: 103.44

🔧 Config 5/27 → filters=32, kernel=3, dropout=0.1, lr=0.001, batch=32
✅ Avg MAE: 34.67 | RMSE: 42.13 | MAPE: 102.34

🔧 Config 6/27 → filters=32, kernel=3, dropout=0.2, lr=0.001, batch=32
✅ Avg MAE: 34.01 | RMSE: 41.49 | MAPE: 106.65

🔧 Config 7/27 → filters=32, kernel=5, dropout=0.0, lr=0.001, batch=32
✅ Avg MAE: 35.52 | RMSE: 42.68 | MAPE: 106.52

🔧 Config 8/27 → filters=32, kernel=5, dropout=0.1, lr=0.001, batch=32
✅ Avg MAE: 35.54 | RMSE: 42.78 | MAPE: 105.66

🔧 Config 9/27 → filters=32, kernel=5, dropout=0.2, lr=0.001, ba

In [ ]:
df_cnn_results = pd.DataFrame(cnn_results)
df_cnn_results = df_cnn_results.sort_values("RMSE").reset_index(drop=True)
df_cnn_results

,filters,kernel,dropout,lr,batch,MAE,RMSE,MAPE
0,32,2,0.1,0.001,32,33.663449,40.987976,103.338586
1,32,2,0.2,0.001,32,34.172721,41.412398,106.291753
2,32,3,0.2,0.001,32,34.007828,41.491944,106.654283
3,32,3,0.0,0.001,32,34.106265,41.537635,103.440944
4,32,5,0.2,0.001,32,34.477077,41.847702,108.403090
5,32,3,0.1,0.001,32,34.669038,42.128338,102.341143
6,64,2,0.0,0.001,32,34.677114,42.208715,114.720858
7,32,2,0.0,0.001,32,35.098770,42.499077,109.169390
8,32,5,0.0,0.001,32,35.522639,42.679479,106.515662
9,64,5,0.2,0.001,32,35.362657,42.741849,111.646540


2-layer CNN

In [ ]:
import torch
import torch.nn as nn

class TwoLayerCNNForecastNet(nn.Module):
    def __init__(self, input_channels, seq_len,
                 num_filters1=64, kernel_size1=3,
                 num_filters2=32, kernel_size2=2,
                 dropout=0.0, output_size=14):
        super().__init__()

        self.conv1 = nn.Conv1d(in_channels=input_channels,
                               out_channels=num_filters1,
                               kernel_size=kernel_size1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)

        self.conv2 = nn.Conv1d(in_channels=num_filters1,
                               out_channels=num_filters2,
                               kernel_size=kernel_size2)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.AdaptiveAvgPool1d(1)  # Global pooling

        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(num_filters2, output_size)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # [batch, channels, seq_len]
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))  # [batch, channels, 1]
        x = x.squeeze(2)  # [batch, channels]
        x = self.dropout(x)
        out = self.fc(x)  # [batch, output_size]
        return out

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import torch
from torch.utils.data import DataLoader, Dataset

def train_one_cnn_fold(X_train, y_train, X_val, y_val,
                       sequence_len=30, num_epochs=50, patience=5,
                       batch_size=32, lr=0.001,
                       num_filters1=64, kernel_size1=3,
                       num_filters2=32, kernel_size2=2,
                       dropout=0.1):

    # 1. Scaling
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train_scaled = scaler_X.fit_transform(X_train)
    X_val_scaled = scaler_X.transform(X_val)

    y_train_scaled = scaler_y.fit_transform(y_train)
    y_val_scaled = scaler_y.transform(y_val)

    # 2. Create CNN sequences
    X_train_seq, y_train_seq = create_lstm_input(X_train_scaled, y_train_scaled, sequence_len)
    X_val_seq, y_val_seq = create_lstm_input(X_val_scaled, y_val_scaled, sequence_len)

    if X_val_seq.shape[0] == 0:
        print("Skipping fold: Not enough validation sequences.")
        return None, None, None

    # 3. Dataset & Dataloader
    train_loader = DataLoader(LSTMForecastDataset(X_train_seq, y_train_seq), batch_size=batch_size, shuffle=False)
    val_loader = DataLoader(LSTMForecastDataset(X_val_seq, y_val_seq), batch_size=batch_size, shuffle=False)

    # 4. Model setup
    model = TwoLayerCNNForecastNet(
        input_channels=X_train_seq.shape[2],
        seq_len=sequence_len,
        num_filters1=num_filters1,
        kernel_size1=kernel_size1,
        num_filters2=num_filters2,
        kernel_size2=kernel_size2,
        dropout=dropout,
        output_size=y_train_seq.shape[1]
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None

    # 5. Training loop
    for epoch in range(num_epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)

            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            val_preds = torch.cat([model(xb.to(device)) for xb, _ in val_loader])
            val_targets = torch.cat([yb.to(device) for _, yb in val_loader])
            val_loss = criterion(val_preds, val_targets)

        if val_loss.item() < best_val_loss - 1e-4:
            best_val_loss = val_loss.item()
            best_model_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break

    # 6. Load best model
    if best_model_state:
        model.load_state_dict(best_model_state)

    # 7. Final evaluation
    model.eval()
    with torch.no_grad():
        X_val_tensor = torch.tensor(X_val_seq, dtype=torch.float32).to(device)
        preds_scaled = model(X_val_tensor).cpu().numpy()
        targets_scaled = y_val_seq

    preds = scaler_y.inverse_transform(preds_scaled)
    targets = scaler_y.inverse_transform(targets_scaled)

    # 8. Metrics
    mae = mean_absolute_error(targets.flatten(), preds.flatten())
    rmse = np.sqrt(mean_squared_error(targets.flatten(), preds.flatten()))
    mape = mean_absolute_percentage_error(targets.flatten(), preds.flatten()) * 100

    return mae, rmse, mape

In [ ]:
sequence_len = 30
horizon = 14
initial_train_size = 1095

mae_scores = []
rmse_scores = []
mape_scores = []

for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
    expanding_window_cv(X, y, initial_train_size=initial_train_size, horizon=horizon, step=horizon, sequence_len=sequence_len)
):
    print(f"\n Fold {fold+1}")
    mae, rmse, mape = train_one_cnn_fold(
        X_tr, y_tr, X_val, y_val,
        sequence_len=sequence_len,
        num_epochs=50,
        patience=5,
        batch_size=32,
        lr=0.001,
        num_filters1=64,
        kernel_size1=3,
        num_filters2=32,
        kernel_size2=2,
        dropout=0.2
    )

    if mae is None:
        continue  # skip invalid fold

    print(f" MAE: {mae:.4f} | RMSE: {rmse:.4f} | MAPE: {mape:.4f}")
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    mape_scores.append(mape)

# --- 7. Final CV Results ---
print("\n Final CNN CV Results:")
print(f"Average MAE:  {np.mean(mae_scores):.4f}")
print(f"Average RMSE: {np.mean(rmse_scores):.4f}")
print(f"Average MAPE: {np.mean(mape_scores):.4f}")


🔁 Fold 1
  📉 MAE: 10.8113 | RMSE: 12.4097 | MAPE: 40.24

🔁 Fold 2
  📉 MAE: 11.1573 | RMSE: 12.5667 | MAPE: 62.22

🔁 Fold 3
  📉 MAE: 7.4514 | RMSE: 9.2816 | MAPE: 50.17

🔁 Fold 4
  📉 MAE: 5.0457 | RMSE: 6.2477 | MAPE: 14.15

🔁 Fold 5
  📉 MAE: 5.4792 | RMSE: 7.5216 | MAPE: 20.43

🔁 Fold 6
  📉 MAE: 6.3125 | RMSE: 8.1016 | MAPE: 24.61

🔁 Fold 7
  📉 MAE: 9.3679 | RMSE: 10.6209 | MAPE: 25.60

🔁 Fold 8
  📉 MAE: 10.9291 | RMSE: 11.8024 | MAPE: 24.61

🔁 Fold 9
  📉 MAE: 10.2448 | RMSE: 11.4666 | MAPE: 22.79

🔁 Fold 10
  📉 MAE: 10.4901 | RMSE: 11.3790 | MAPE: 21.67

🔁 Fold 11
  📉 MAE: 7.8011 | RMSE: 8.4752 | MAPE: 14.54

🔁 Fold 12
  📉 MAE: 6.8545 | RMSE: 8.2418 | MAPE: 12.25

🔁 Fold 13
  📉 MAE: 7.0330 | RMSE: 8.5626 | MAPE: 12.01

🔁 Fold 14
  📉 MAE: 7.2098 | RMSE: 8.6800 | MAPE: 11.99

🔁 Fold 15
  📉 MAE: 14.9396 | RMSE: 16.9888 | MAPE: 28.73

🔁 Fold 16
  📉 MAE: 12.3277 | RMSE: 15.4036 | MAPE: 37.78

🔁 Fold 17
  📉 MAE: 12.3386 | RMSE: 14.8800 | MAPE: 34.21

🔁 Fold 18
  📉 MAE: 9.5719 | RMSE: 12.35

Hyperparameter tunning

In [ ]:
import itertools

# Define hyperparameter options
num_filters1_list = [64, 128]
num_filters2_list = [32, 64]
kernel_size1_list = [2, 3]
kernel_size2_list = [2, 3]
dropouts = [0.0, 0.1, 0.2]
learning_rates = [0.001]
batch_sizes = [32]

# Create full grid of combinations
param_grid = list(itertools.product(
    num_filters1_list,
    num_filters2_list,
    kernel_size1_list,
    kernel_size2_list,
    dropouts,
    learning_rates,
    batch_sizes
))

print(f"Total combinations: {len(param_grid)}")

🔍 Total combinations: 48


In [ ]:
results = []

sequence_len = 30
horizon = 14
initial_train_size = 1095
step = horizon

for i, (nf1, nf2, ks1, ks2, dropout, lr, batch_size) in enumerate(param_grid):
    print(f"\n🔧 Config {i+1}/{len(param_grid)} → "
          f"nf1={nf1}, nf2={nf2}, ks1={ks1}, ks2={ks2}, dropout={dropout}, lr={lr}, batch={batch_size}")

    mae_scores, rmse_scores, mape_scores = [], [], []

    for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
        expanding_window_cv(X, y, initial_train_size=initial_train_size,
                            horizon=horizon, step=step, sequence_len=sequence_len)
    ):
        mae, rmse, mape = train_one_cnn_fold(
            X_tr, y_tr, X_val, y_val,
            sequence_len=sequence_len,
            num_epochs=50,
            patience=5,
            batch_size=batch_size,
            lr=lr,
            num_filters1=nf1,
            num_filters2=nf2,
            kernel_size1=ks1,
            kernel_size2=ks2,
            dropout=dropout
        )

        if mae is None:
            continue

        mae_scores.append(mae)
        rmse_scores.append(rmse)
        mape_scores.append(mape)

    # Average fold results
    avg_mae = np.mean(mae_scores)
    avg_rmse = np.mean(rmse_scores)
    avg_mape = np.mean(mape_scores)

    print(f"Avg MAE: {avg_mae:.4f} | RMSE: {avg_rmse:.4f} | MAPE: {avg_mape:.4f}")

    results.append({
        "filters1": nf1,
        "filters2": nf2,
        "kernel1": ks1,
        "kernel2": ks2,
        "dropout": dropout,
        "lr": lr,
        "batch": batch_size,
        "MAE": avg_mae,
        "RMSE": avg_rmse,
        "MAPE": avg_mape
    })


🔧 Config 1/48 → nf1=64, nf2=32, ks1=2, ks2=2, dropout=0.0, lr=0.001, batch=32
📊 Avg MAE: 33.9775 | RMSE: 41.3638 | MAPE: 106.6675

🔧 Config 2/48 → nf1=64, nf2=32, ks1=2, ks2=2, dropout=0.1, lr=0.001, batch=32
📊 Avg MAE: 33.8449 | RMSE: 41.2838 | MAPE: 107.7709

🔧 Config 3/48 → nf1=64, nf2=32, ks1=2, ks2=2, dropout=0.2, lr=0.001, batch=32
📊 Avg MAE: 33.6322 | RMSE: 40.9703 | MAPE: 107.1844

🔧 Config 4/48 → nf1=64, nf2=32, ks1=2, ks2=3, dropout=0.0, lr=0.001, batch=32
📊 Avg MAE: 34.3882 | RMSE: 41.8156 | MAPE: 109.5020

🔧 Config 5/48 → nf1=64, nf2=32, ks1=2, ks2=3, dropout=0.1, lr=0.001, batch=32
📊 Avg MAE: 33.7267 | RMSE: 41.1032 | MAPE: 107.4888

🔧 Config 6/48 → nf1=64, nf2=32, ks1=2, ks2=3, dropout=0.2, lr=0.001, batch=32
📊 Avg MAE: 33.7496 | RMSE: 41.0835 | MAPE: 105.9381

🔧 Config 7/48 → nf1=64, nf2=32, ks1=3, ks2=2, dropout=0.0, lr=0.001, batch=32
📊 Avg MAE: 34.3031 | RMSE: 41.6557 | MAPE: 108.1645

🔧 Config 8/48 → nf1=64, nf2=32, ks1=3, ks2=2, dropout=0.1, lr=0.001, batch=32
📊 Av

In [ ]:
import pandas as pd

df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by="RMSE").reset_index(drop=True)

# Display best configs
df_results

,filters1,filters2,kernel1,kernel2,dropout,lr,batch,MAE,RMSE,MAPE
0,128,64,2,3,0.2,0.001,32,32.497195,39.859975,98.387844
1,128,32,3,2,0.2,0.001,32,32.875210,40.139777,102.137753
2,128,32,2,3,0.0,0.001,32,33.156406,40.650717,101.662257
3,64,32,3,3,0.2,0.001,32,33.458933,40.706932,102.390350
4,128,64,2,2,0.1,0.001,32,33.569576,40.864505,104.268514
5,64,32,2,2,0.2,0.001,32,33.632216,40.970308,107.184378
6,64,32,3,2,0.1,0.001,32,33.629762,41.027502,105.026401
7,64,32,3,2,0.2,0.001,32,33.759234,41.028450,103.671009
8,128,32,3,3,0.0,0.001,32,33.812513,41.076084,104.290589
9,64,32,2,3,0.2,0.001,32,33.749580,41.083515,105.938059


TCN

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Chomp1d(nn.Module):
    """Removes padding from the end to maintain causality."""
    def __init__(self, chomp_size):
        super().__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size]  # Remove last chomp_size elements


class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, dilation, padding, dropout):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                               stride=stride, padding=padding, dilation=dilation)
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                               stride=stride, padding=padding, dilation=dilation)
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.conv1(x)
        out = self.chomp1(out)
        out = self.relu1(out)
        out = self.dropout1(out)

        out = self.conv2(out)
        out = self.chomp2(out)
        out = self.relu2(out)
        out = self.dropout2(out)

        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=3, dropout=0.2):
        """
        num_inputs: number of input features
        num_channels: list of output channels for each TCN layer (e.g. [64, 64])
        """
        super().__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1,
                                     dilation=dilation_size, padding=(kernel_size-1)*dilation_size,
                                     dropout=dropout)]
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        # x: [batch, seq_len, features]
        x = x.permute(0, 2, 1)  # [batch, features, seq_len]
        out = self.network(x)
        out = out[:, :, -1]  # Use last timestep
        return out


class TCNForecastNet(nn.Module):
    def __init__(self, input_size, num_channels=[64, 64], kernel_size=3, dropout=0.2, output_size=14):
        super().__init__()
        self.tcn = TemporalConvNet(num_inputs=input_size, num_channels=num_channels,
                                   kernel_size=kernel_size, dropout=dropout)
        self.fc = nn.Linear(num_channels[-1], output_size)

    def forward(self, x):
        out = self.tcn(x)
        out = self.fc(out)
        return out

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset

class TCNForecastDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


def create_tcn_input(X_scaled, y_scaled, sequence_len=30):
    X_seq = []
    y_seq = []
    for i in range(sequence_len, len(X_scaled)):
        X_window = X_scaled[i-sequence_len:i]
        y_target = y_scaled[i]  # shape: (14,)
        X_seq.append(X_window)
        y_seq.append(y_target)
    return np.array(X_seq), np.array(y_seq)


def train_one_tcn_fold(X_train, y_train, X_val, y_val,
                       sequence_len=30, num_epochs=50, patience=5,
                       batch_size=32, lr=0.001,
                       num_channels=[64, 64], kernel_size=3, dropout=0.2):

    # --- 1. Scaling ---
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train_scaled = scaler_X.fit_transform(X_train)
    X_val_scaled = scaler_X.transform(X_val)

    y_train_scaled = scaler_y.fit_transform(y_train)
    y_val_scaled = scaler_y.transform(y_val)

    # --- 2. Create TCN inputs ---
    X_train_seq, y_train_seq = create_tcn_input(X_train_scaled, y_train_scaled, sequence_len)
    X_val_seq, y_val_seq = create_tcn_input(X_val_scaled, y_val_scaled, sequence_len)

    if len(X_val_seq) == 0:
        print("Skipping fold: not enough validation sequences.")
        return None, None, None

    # --- 3. Dataloaders ---
    train_loader = DataLoader(
        TCNForecastDataset(X_train_seq, y_train_seq), batch_size=batch_size, shuffle=False
    )
    val_loader = DataLoader(
        TCNForecastDataset(X_val_seq, y_val_seq), batch_size=batch_size, shuffle=False
    )

    # --- 4. Model Setup ---
    model = TCNForecastNet(
        input_size=X_train_seq.shape[2],
        num_channels=num_channels,
        kernel_size=kernel_size,
        dropout=dropout,
        output_size=y_train.shape[1]  # usually 14
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.MSELoss()

    # --- 5. Early stopping setup ---
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None

    for epoch in range(num_epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            val_preds = torch.cat([model(xb.to(device)) for xb, _ in val_loader])
            val_targets = torch.cat([yb.to(device) for _, yb in val_loader])
            val_loss = criterion(val_preds, val_targets)

        if val_loss.item() < best_val_loss - 1e-4:
            best_val_loss = val_loss.item()
            best_model_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break

    # --- 6. Load best model ---
    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    # --- 7. Final evaluation ---
    model.eval()
    with torch.no_grad():
        X_val_tensor = torch.tensor(X_val_seq, dtype=torch.float32).to(device)
        preds_scaled = model(X_val_tensor).cpu().numpy()

    targets_scaled = y_val_seq
    preds = scaler_y.inverse_transform(preds_scaled)
    targets = scaler_y.inverse_transform(targets_scaled)

    mae = mean_absolute_error(targets.flatten(), preds.flatten())
    rmse = np.sqrt(mean_squared_error(targets.flatten(), preds.flatten()))
    mape = mean_absolute_percentage_error(targets.flatten(), preds.flatten()) * 100

    return mae, rmse, mape

In [ ]:
sequence_len = 30
horizon = 14
initial_train_size = 1095

mae_scores = []
rmse_scores = []
mape_scores = []

for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
    expanding_window_cv(X, y, initial_train_size=initial_train_size, horizon=horizon, step=horizon, sequence_len=sequence_len)
):
    print(f"\nFold {fold+1}")
    mae, rmse, mape = train_one_tcn_fold(
        X_tr, y_tr, X_val, y_val,
        sequence_len=sequence_len,
        num_epochs=50,
        patience=5,
        batch_size=32,
        lr=0.001,
        num_channels=[64, 64],
        kernel_size=3,
        dropout=0.2
    )

    if mae is None:
        continue

    print(f"  MAE: {mae:.4f} | RMSE: {rmse:.4f} | MAPE: {mape:.4f}")
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    mape_scores.append(mape)

# --- Final Results ---
print("\n Final TCN CV Results:")
print(f"Average MAE:  {np.mean(mae_scores):.4f}")
print(f"Average RMSE: {np.mean(rmse_scores):.4f}")
print(f"Average MAPE: {np.mean(mape_scores):.4f}")


Fold 1
  MAE: 8.8175 | RMSE: 10.6820 | MAPE: 35.9513

Fold 2
  MAE: 8.8393 | RMSE: 10.6083 | MAPE: 52.1615

Fold 3
  MAE: 8.1011 | RMSE: 9.5065 | MAPE: 44.8374

Fold 4
  MAE: 4.8996 | RMSE: 6.1100 | MAPE: 14.6355

Fold 5
  MAE: 5.4180 | RMSE: 7.3444 | MAPE: 20.2983

Fold 6
  MAE: 6.5057 | RMSE: 8.2876 | MAPE: 24.3780

Fold 7
  MAE: 9.2561 | RMSE: 10.2730 | MAPE: 24.5859

Fold 8
  MAE: 8.1772 | RMSE: 9.3998 | MAPE: 19.1793

Fold 9
  MAE: 7.3479 | RMSE: 9.0078 | MAPE: 17.6021

Fold 10
  MAE: 7.7834 | RMSE: 8.9665 | MAPE: 17.0664

Fold 11
  MAE: 6.5112 | RMSE: 7.4065 | MAPE: 12.2486

Fold 12
  MAE: 6.0139 | RMSE: 7.7098 | MAPE: 10.9147

Fold 13
  MAE: 6.6347 | RMSE: 8.0358 | MAPE: 11.5139

Fold 14
  MAE: 7.4766 | RMSE: 8.9077 | MAPE: 12.5203

Fold 15
  MAE: 11.1171 | RMSE: 14.2510 | MAPE: 28.6449

Fold 16
  MAE: 12.9627 | RMSE: 15.8488 | MAPE: 37.4568

Fold 17
  MAE: 13.1779 | RMSE: 15.8476 | MAPE: 33.1182

Fold 18
  MAE: 10.2801 | RMSE: 13.0285 | MAPE: 22.9988

Fold 19
  MAE: 10.9501 | 

Hyperparameter tunning (lr = 0.001)

In [ ]:
from itertools import product

# Grid options
num_channels_list = [
    [64, 64],
    [128, 64],
    [128, 128]
]

kernel_sizes = [2, 3, 5]
dropouts = [0.0, 0.1, 0.2]
learning_rates = [0.001]
batch_sizes = [32]

# Generate combinations
tcn_param_grid = list(product(num_channels_list, kernel_sizes, dropouts, learning_rates, batch_sizes))
print(f"Total combinations: {len(tcn_param_grid)}")

🔍 Total combinations: 27


In [ ]:
tcn_results = []

for i, (channels, kernel_size, dropout, lr, batch_size) in enumerate(tcn_param_grid):
    print(f"\n Config {i+1}/{len(tcn_param_grid)} → channels={channels}, kernel={kernel_size}, dropout={dropout}, lr={lr}, batch={batch_size}")

    mae_scores = []
    rmse_scores = []
    mape_scores = []

    for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
        expanding_window_cv(X, y, initial_train_size=1095, horizon=14, step=14, sequence_len=30)
    ):
        mae, rmse, mape = train_one_tcn_fold(
            X_tr, y_tr, X_val, y_val,
            sequence_len=30,
            num_epochs=50,
            patience=5,
            batch_size=batch_size,
            lr=lr,
            num_channels=channels,
            kernel_size=kernel_size,
            dropout=dropout
        )

        if mae is None:
            continue

        mae_scores.append(mae)
        rmse_scores.append(rmse)
        mape_scores.append(mape)

    # Save results
    tcn_results.append({
        "channels": channels,
        "kernel_size": kernel_size,
        "dropout": dropout,
        "lr": lr,
        "batch_size": batch_size,
        "MAE": np.mean(mae_scores),
        "RMSE": np.mean(rmse_scores),
        "MAPE": np.mean(mape_scores)
    })

    print(f"Avg MAE: {np.mean(mae_scores):.4f} | RMSE: {np.mean(rmse_scores):.4f} | MAPE: {np.mean(mape_scores):.4f}")


🔧 Config 1/27 → channels=[64, 64], kernel=2, dropout=0.0, lr=0.001, batch=32
→ Avg MAE: 30.2883 | RMSE: 37.7419 | MAPE: 95.4350

🔧 Config 2/27 → channels=[64, 64], kernel=2, dropout=0.1, lr=0.001, batch=32
→ Avg MAE: 30.1341 | RMSE: 37.5239 | MAPE: 94.3021

🔧 Config 3/27 → channels=[64, 64], kernel=2, dropout=0.2, lr=0.001, batch=32
→ Avg MAE: 30.0400 | RMSE: 37.4928 | MAPE: 93.3407

🔧 Config 4/27 → channels=[64, 64], kernel=3, dropout=0.0, lr=0.001, batch=32
→ Avg MAE: 29.9895 | RMSE: 37.4984 | MAPE: 92.6713

🔧 Config 5/27 → channels=[64, 64], kernel=3, dropout=0.1, lr=0.001, batch=32
→ Avg MAE: 29.8698 | RMSE: 37.2964 | MAPE: 93.2529

🔧 Config 6/27 → channels=[64, 64], kernel=3, dropout=0.2, lr=0.001, batch=32
→ Avg MAE: 29.6635 | RMSE: 37.0594 | MAPE: 92.3209

🔧 Config 7/27 → channels=[64, 64], kernel=5, dropout=0.0, lr=0.001, batch=32
→ Avg MAE: 29.7391 | RMSE: 37.1541 | MAPE: 91.8287

🔧 Config 8/27 → channels=[64, 64], kernel=5, dropout=0.1, lr=0.001, batch=32
→ Avg MAE: 30.1389 

In [ ]:
import pandas as pd

df_tcn = pd.DataFrame(tcn_results)
df_tcn = df_tcn.sort_values(by="RMSE").reset_index(drop=True)
df_tcn

,channels,kernel_size,dropout,lr,batch_size,MAE,RMSE,MAPE
0,"[128, 128]",5,0.2,0.001,32,29.010372,36.261627,90.503628
1,"[128, 128]",3,0.2,0.001,32,29.581293,36.925982,92.261882
2,"[64, 64]",5,0.2,0.001,32,29.564508,36.935240,91.407405
3,"[64, 64]",3,0.2,0.001,32,29.663507,37.059407,92.320876
4,"[64, 64]",5,0.0,0.001,32,29.739125,37.154115,91.828736
5,"[128, 128]",2,0.0,0.001,32,29.777122,37.207161,93.380802
6,"[64, 64]",3,0.1,0.001,32,29.869814,37.296391,93.252879
7,"[128, 128]",3,0.0,0.001,32,29.890747,37.333907,93.582211
8,"[64, 64]",5,0.1,0.001,32,30.138905,37.419350,92.222743
9,"[64, 64]",2,0.2,0.001,32,30.039987,37.492814,93.340683


Hyperparameter tunning (lr = 0.0005)

In [ ]:
from itertools import product

# Grid options
num_channels_list = [
    [64, 64],
    [128, 64],
    [128, 128]
]

kernel_sizes = [2, 3, 5]
dropouts = [0.0, 0.1, 0.2]
learning_rates = [0.0005]
batch_sizes = [32]

# Generate combinations
tcn_param_grid = list(product(num_channels_list, kernel_sizes, dropouts, learning_rates, batch_sizes))
print(f"Total combinations: {len(tcn_param_grid)}")

🔍 Total combinations: 27


In [ ]:
tcn_results = []

for i, (channels, kernel_size, dropout, lr, batch_size) in enumerate(tcn_param_grid):
    print(f"\n Config {i+1}/{len(tcn_param_grid)} → channels={channels}, kernel={kernel_size}, dropout={dropout}, lr={lr}, batch={batch_size}")

    mae_scores = []
    rmse_scores = []
    mape_scores = []

    for fold, (X_tr, y_tr, X_val, y_val) in enumerate(
        expanding_window_cv(X, y, initial_train_size=1095, horizon=14, step=14, sequence_len=30)
    ):
        mae, rmse, mape = train_one_tcn_fold(
            X_tr, y_tr, X_val, y_val,
            sequence_len=30,
            num_epochs=50,
            patience=5,
            batch_size=batch_size,
            lr=lr,
            num_channels=channels,
            kernel_size=kernel_size,
            dropout=dropout
        )

        if mae is None:
            continue

        mae_scores.append(mae)
        rmse_scores.append(rmse)
        mape_scores.append(mape)

    # Save results
    tcn_results.append({
        "channels": channels,
        "kernel_size": kernel_size,
        "dropout": dropout,
        "lr": lr,
        "batch_size": batch_size,
        "MAE": np.mean(mae_scores),
        "RMSE": np.mean(rmse_scores),
        "MAPE": np.mean(mape_scores)
    })

    print(f"→ Avg MAE: {np.mean(mae_scores):.4f} | RMSE: {np.mean(rmse_scores):.4f} | MAPE: {np.mean(mape_scores):.4f}")


🔧 Config 1/27 → channels=[64, 64], kernel=2, dropout=0.0, lr=0.0005, batch=32
→ Avg MAE: 30.6086 | RMSE: 38.1115 | MAPE: 94.7117

🔧 Config 2/27 → channels=[64, 64], kernel=2, dropout=0.1, lr=0.0005, batch=32
→ Avg MAE: 29.9620 | RMSE: 37.4246 | MAPE: 93.7656

🔧 Config 3/27 → channels=[64, 64], kernel=2, dropout=0.2, lr=0.0005, batch=32
→ Avg MAE: 30.0178 | RMSE: 37.3518 | MAPE: 94.3049

🔧 Config 4/27 → channels=[64, 64], kernel=3, dropout=0.0, lr=0.0005, batch=32
→ Avg MAE: 30.2330 | RMSE: 37.6867 | MAPE: 93.2914

🔧 Config 5/27 → channels=[64, 64], kernel=3, dropout=0.1, lr=0.0005, batch=32
→ Avg MAE: 29.8627 | RMSE: 37.3141 | MAPE: 94.0388

🔧 Config 6/27 → channels=[64, 64], kernel=3, dropout=0.2, lr=0.0005, batch=32
→ Avg MAE: 29.5401 | RMSE: 36.8300 | MAPE: 93.0457

🔧 Config 7/27 → channels=[64, 64], kernel=5, dropout=0.0, lr=0.0005, batch=32
→ Avg MAE: 30.4781 | RMSE: 37.9067 | MAPE: 94.5364

🔧 Config 8/27 → channels=[64, 64], kernel=5, dropout=0.1, lr=0.0005, batch=32
→ Avg MAE: 

In [ ]:
import pandas as pd

df_tcn = pd.DataFrame(tcn_results)
df_tcn = df_tcn.sort_values(by="RMSE").reset_index(drop=True)
df_tcn

,channels,kernel_size,dropout,lr,batch_size,MAE,RMSE,MAPE
0,"[64, 64]",5,0.2,0.0005,32,29.521908,36.801219,92.661470
1,"[64, 64]",3,0.2,0.0005,32,29.540073,36.830018,93.045734
2,"[128, 128]",3,0.1,0.0005,32,29.567380,36.913412,92.987458
3,"[128, 128]",5,0.1,0.0005,32,29.663175,37.003283,93.272773
4,"[128, 128]",5,0.2,0.0005,32,29.645477,37.015112,93.681235
5,"[128, 128]",2,0.1,0.0005,32,29.768491,37.139857,93.349502
6,"[128, 128]",2,0.2,0.0005,32,29.819751,37.249475,93.604952
7,"[64, 64]",3,0.1,0.0005,32,29.862674,37.314095,94.038797
8,"[128, 128]",3,0.2,0.0005,32,29.934321,37.317935,92.663935
9,"[64, 64]",5,0.1,0.0005,32,29.971523,37.323805,92.585483
